# Air Quality Analysis

This notebook presents a comprehensive analysis of air quality data, including advanced feature engineering, predictive modeling, clustering, and anomaly detection techniques.

## Import Libraries

In [ ]:
# Basic data manipulation and visualization
import pandas as pd
import numpy as np
# Interactive visualization with Plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = "notebook"  # For Jupyter notebook display



# Time series analysis
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA

# Machine learning - preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score

# Machine learning - regression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Machine learning - classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, auc

# System utilities
import os
import warnings
warnings.filterwarnings('ignore')

### Creating Output Directories

Setting up directory structure for organizing analysis outputs.

In [ ]:
# Create directories for outputs
os.makedirs('Plotly_Plots', exist_ok=True)
os.makedirs('Plotly_Plots/data', exist_ok=True)
os.makedirs('Plotly_Plots/EDA_plots', exist_ok=True)
os.makedirs('Plotly_Plots/preprocessing_plots', exist_ok=True)
os.makedirs('Plotly_Plots/correlation_analysis_plots', exist_ok=True)
os.makedirs('Plotly_Plots/time_series_analysis_plots', exist_ok=True)
os.makedirs('Plotly_Plots/modelling_analysis_plots', exist_ok=True)

# Phase 1: Data Loading and Initial Examination

In this phase, we load the air quality dataset and perform initial examination to understand its structure, variables, and potential issues.

### 1.1 Loading the Dataset

Loading the Air Quality UCI dataset from Excel file.

In [ ]:
print("Phase 1: Data Loading and Initial Examination")

In [ ]:
# Load the dataset
airquality = pd.read_excel('Dataset/AirQualityUCI.xlsx')

### 1.2 Previewing the Dataset

Examining the first few rows to understand the data structure.

In [ ]:
airquality.head()

### 1.3 Dataset Dimensions

Checking the number of rows and columns in the dataset.

In [ ]:
print(f"Dataset shape: {airquality.shape}")

### 1.4 Column Names

Listing all variables in the dataset.

In [ ]:
print("\nColumn names:")
print(airquality.columns.tolist())

### 1.5 Data Information

Examining data types and non-null counts for each column.

In [ ]:
airquality.info()

### 1.6 Data Types

Checking the data type of each column.

In [ ]:
print("\nData types:")
print(airquality.dtypes)

### 1.7 Summary Statistics

Calculating descriptive statistics for numerical variables.

In [ ]:
airquality.describe()

### 1.8 Missing Values Analysis

Checking for null values in the dataset.

In [ ]:
airquality.isnull().sum()

### 1.9 Special Missing Values (-200)

Identifying columns with -200 values, which represent missing data in this dataset.

In [ ]:
# Check for missing values (represented as -200)
print("\nChecking for -200 values (missing data):")
for col in airquality.columns:
    if isinstance(airquality[col].min(), (int, float)) and airquality[col].min() == -200:
        print(f"{col} has -200 values: {(airquality[col] == -200).sum()} ({(airquality[col] == -200).sum()/len(airquality)*100:.2f}%)")

### 1.10 Replacing Special Missing Values

Converting -200 values to NaN for proper handling of missing data.

In [ ]:
# Replace -200 with NaN
airquality = airquality.copy()
for col in airquality.columns:
    if airquality[col].dtype != 'datetime64[ns]' and airquality[col].dtype != 'object':
        airquality[col] = airquality[col].replace(-200, np.nan)

In [ ]:
# Check for missing values (represented as -200)
print("\nChecking for -200 values (missing data):")
for col in airquality.columns:
    if isinstance(airquality[col].min(), (int, float)) and airquality[col].min() == -200:
        print(f"{col} has -200 values: {(airquality[col] == -200).sum()} ({(airquality[col] == -200).sum()/len(airquality)*100:.2f}%)")

### 1.11 Checking for Duplicates

Identifying duplicate rows in the dataset.

In [ ]:
# Check for duplicates
duplicates = airquality.duplicated().sum()
print(f"\nNumber of duplicate rows: {duplicates}")

### 1.12 Verifying Missing Values

Rechecking missing values after replacing -200 with NaN.

In [ ]:
airquality.isnull().sum()

In [ ]:
# Create subplots: 3 rows, 2 cols. Define specs for different types
fig = make_subplots(
    rows=3, cols=2,
    specs=[[{"type": "domain", "colspan": 2}, None],
           [{"type": "xy"}, {"type": "domain"}],
           [{"type": "xy", "colspan": 2}, None]],
    subplot_titles=(
        None, # Row 1 title (handled by annotation)
        'Missing Data (%)', 'Variable Categories',
        'Dataset Time Coverage'
    ),
    row_heights=[0.2, 0.4, 0.4],
    vertical_spacing=0.15
)

# --- Subplot 1: Dataset Overview (using annotations) ---
# --- Subplot 1: Dataset Overview (using annotations) ---
fig.add_annotation(
    xref="paper", yref="paper",
    x=0.5, y=0.95, 
    text='<b>Air Quality UCI Dataset</b>',
    showarrow=False, font=dict(size=20), align='center'
)
fig.add_annotation(
    xref="paper", yref="paper",
    x=0.5, y=0.88,
    text='Hourly measurements from an Italian city (March 2004 - February 2005)',
    showarrow=False, font=dict(size=14), align='center'
)
fig.add_annotation(
    xref="paper", yref="paper",
    x=0.5, y=0.81,
    text=f'{airquality.shape[0]} hourly records, {airquality.shape[1]} variables',
    showarrow=False, font=dict(size=12), align='center'
)


# --- Subplot 2: Data Completeness (Missing %) --- (Row 2, Col 1)
missing_data = airquality.isnull().sum().sort_values(ascending=False)
missing_percent = missing_data / len(airquality) * 100
missing_df = pd.DataFrame({'Missing Count': missing_data, 'Missing Percent': missing_percent})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Percent') # Sort for horizontal bar
fig.add_trace(
    go.Bar(
        y=missing_df.index,
        x=missing_df['Missing Percent'],
        orientation='h',
        name='Missing %',
        text=missing_df['Missing Percent'].apply(lambda x: f'{x:.1f}%'),
        textposition='outside',
        hoverinfo='y+x',
        hovertemplate='<b>%{y}</b><br>Missing: %{x:.1f}%<extra></extra>',
        marker_color='steelblue'
    ),
    row=2, col=1
)

# --- Subplot 3: Variable Categories (Pie Chart) --- (Row 2, Col 2)
categories = ['Air pollutants', 'Sensor readings', 'Environmental']
counts = [5, 5, 3]  # CO, NOx, NO2, NMHC, C6H6 | PT08.S1-5 | T, RH, AH
colors = ['#ff9999', '#66b3ff', '#99ff99'] # Keep original colors
fig.add_trace(
    go.Pie(
        labels=categories,
        values=counts,
        name='Categories',
        marker_colors=colors,
        hole=0.3, 
        hoverinfo='label+percent+value',
        textinfo='percent+label',
        insidetextorientation='radial'
    ),
    row=2, col=2
)

# --- Subplot 4: Time Span Visualization --- (Row 3, Col 1, colspan=2)
date_range = pd.to_datetime(airquality['Date']).unique() # Get unique dates
fig.add_trace(
    go.Scatter(
        x=date_range,
        y=np.ones(len(date_range)),
        mode='markers',
        marker=dict(symbol='line-ns-open', size=8, color='blue'), 
        name='Data Points',
        hoverinfo='x',
        hovertemplate='%{x|%Y-%m-%d}<extra></extra>'
    ),
    row=3, col=1
)
fig.update_yaxes(showticklabels=False, showgrid=False, zeroline=False, row=3, col=1)
fig.update_xaxes(title_text='Date', row=3, col=1, gridcolor='lightgrey', gridwidth=1)

# --- Layout Updates ---
fig.update_layout(
    title_text='Air Quality Dataset Overview', title_x=0.5,
    height=800, 
    showlegend=False,
    # Add source text via annotation below plot area
    annotations=list(fig.layout.annotations) + [
        go.layout.Annotation(
            text="Source: UCI Machine Learning Repository | Savona, Italy - De Vito et al. (2008) | Features include ground truth measurements (GT) and sensor responses",
            showarrow=False,
            xref='paper', yref='paper',
            x=0.5, y=-0.12, # Adjust y position to be below plot
            xanchor='center', yanchor='top',
            font=dict(size=8.5)
        )
    ],
    margin=dict(l=50, r=50, t=100, b=120) # Increase bottom margin for source text
)

# Update subplot titles font size
for annotation in fig.layout.annotations:
    # Check if it's a subplot title generated by make_subplots
    if annotation.text in ['Missing Data (%)', 'Variable Categories', 'Dataset Time Coverage']:
        annotation.font.size = 14

fig.show()

fig.write_image('Plotly_Plots/data/dataset_description_plotly.png')



# Phase 2: Exploratory Data Analysis (EDA)

In this phase, we perform a deeper investigation of the dataset through visualizations and statistical analyses to understand distributions, patterns, and relationships.

## Phase 1 Summary

In this initial phase, we successfully loaded the air quality dataset and performed a preliminary examination to understand its structure, variables, and potential issues. The dataset contains hourly measurements of various pollutants and environmental factors over a one-year period, providing a rich source of information for analysis.

Key findings from this phase include:

- The dataset contains 9,357 hourly records with 13 variables, including pollutant concentrations, temperature, and humidity measurements.
- Several variables have missing values, particularly NMHC(GT) with approximately 90% missing data.
- Missing values are represented as -200 in the original dataset and have been converted to NaN for proper handling.
- No duplicate rows were found in the dataset.
- The dataset includes both ground truth measurements (GT) and sensor responses (PT08.S1 to PT08.S5).

This initial examination provides the foundation for the more detailed analyses in subsequent phases. The identified data quality issues, particularly the missing values, will need to be addressed in the preprocessing phase.

# Phase 2: Exploratory Data Analysis (EDA)

### 2.1 Missing Values Visualization

Creating a heatmap to visualize the pattern of missing values in the dataset.

In [ ]:
print("\nPhase 2: Exploratory Data Analysis")

In [ ]:
df_clean = airquality.copy()

In [ ]:
# Ensure df_clean is defined and pandas (pd) is imported
# df_clean = airquality.copy() # If needed

# --- Plotly Missing Values Bar Chart ---
missing_percent = df_clean.isna().mean().sort_values(ascending=False) * 100
missing_percent_df = missing_percent[missing_percent > 0].reset_index()
missing_percent_df.columns = ['Column', 'Percentage']

fig = px.bar(
    missing_percent_df,
    x='Column',
    y='Percentage',
    title='Percentage of Missing Values by Column',
    labels={'Percentage': 'Missing Values (%)', 'Column': 'Columns'},
    text=missing_percent_df['Percentage'].apply(lambda x: f'{x:.1f}%'),
    height=500
)

fig.update_traces(
    textposition='outside',
    marker_color='steelblue',
    hovertemplate='<b>%{x}</b><br>Missing: %{y:.1f}%<extra></extra>'
)

fig.update_layout(
    xaxis_tickangle=-45,
    yaxis_gridcolor='lightgrey',
    yaxis_gridwidth=1,
    bargap=0.2,
    yaxis_title='Missing Values (%)', # Ensure y-axis label is set
    xaxis_title='Columns' # Ensure x-axis label is set
)

fig.show()

fig.write_image('Plotly_Plots/EDA_plots/dataset_description_plotly.png', scale=2)


### 2.2 Distribution Analysis

Examining the distribution of key variables using histograms.

In [ ]:
# Ensure df_clean, np, pd are defined/imported
# df_clean = airquality.copy() # If needed

# --- Plotly Distribution Histograms ---
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()

# Create subplots
n_cols = 3
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols
fig = make_subplots(rows=n_rows, cols=n_cols, subplot_titles=numeric_cols)

for i, col in enumerate(numeric_cols):
    row = (i // n_cols) + 1
    col_num = (i % n_cols) + 1
    # Use go.Histogram for subplots
    fig.add_trace(
        go.Histogram(
            x=df_clean[col].dropna(),
            name=col,
            showlegend=False,
            hovertemplate=f'<b>{col}</b><br>Value Range: %{{x}}<br>Count: %{{y}}<extra></extra>'
            # histnorm='probability density' # Add if KDE was important
        ),
        row=row, col=col_num
    )

fig.update_layout(
    title_text='Distribution of Numeric Variables',
    height=n_rows * 300, 
    showlegend=False
)

# Update subplot titles font size
for annotation in fig.layout.annotations:
    annotation.font.size = 12

fig.show()

fig.write_image('Plotly_Plots/EDA_plots/histograms_plotly.png', scale=1.5, width=1000, height=n_rows*300)


### 2.3 Outlier Analysis

Identifying outliers in key variables using box plots.

In [ ]:
# Ensure df_clean, numeric_cols, pd are defined/imported
# df_clean = airquality.copy() # If needed
# numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()

# --- Plotly Outlier Box Plots ---
# Melt the dataframe for Plotly Express
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist() # Ensure numeric_cols is defined here
df_clean_melt = pd.melt(df_clean[numeric_cols], var_name='variable', value_name='value')

fig = px.box(
    df_clean_melt,
    x='variable',
    y='value',
    title='Box Plots of Numeric Variables',
    points='outliers', 
    height=600 
)

fig.update_layout(
    xaxis_title=None, 
    yaxis_title='Value',
    xaxis_tickangle=-90 
)

fig.update_traces(
    hovertemplate='<b>%{x}</b><br>Value: %{y}<br>Variable: %{x}<extra></extra>', # Improved hover
)

fig.show()

fig.write_image('Plotly_Plots/EDA_plots/boxplots_plotly.png', scale=2)


### 2.4 Temporal Patterns

Analyzing how variables change over time to identify trends and patterns.

In [ ]:
# Create time series plots for key pollutants
# First, ensure datetime format
df_clean['DateTime'] = pd.to_datetime(df_clean['Date'].astype(str) + ' ' + df_clean['Time'].astype(str))
df_clean = df_clean.set_index('DateTime')

In [ ]:
# Ensure df_clean is defined, has DateTime index, and pd, go are imported
# df_clean = airquality.copy()
# df_clean['DateTime'] = pd.to_datetime(df_clean['Date'].astype(str) + ' ' + df_clean['Time'].astype(str))
# df_clean = df_clean.set_index('DateTime') # Ensure this is done before this cell

# --- Plotly Pollutant Time Series ---
pollutants = ['CO(GT)', 'NOx(GT)', 'NO2(GT)']

# Resample data first
df_daily_mean = df_clean[pollutants].resample('D').mean()

# Create subplots
fig = make_subplots(rows=len(pollutants), cols=1, shared_xaxes=True,
                    subplot_titles=[f'Daily Average {p}' for p in pollutants])

for i, pollutant in enumerate(pollutants):
    fig.add_trace(
        go.Scatter(
            x=df_daily_mean.index,
            y=df_daily_mean[pollutant],
            mode='lines',
            name=pollutant,
            showlegend=False,
            hovertemplate=f'<b>{pollutant}</b><br>Date: %{{x|%Y-%m-%d}}<br>Concentration: %{{y:.2f}}<extra></extra>'
        ),
        row=i + 1, col=1
    )
    fig.update_yaxes(title_text='Concentration', row=i + 1, col=1)

fig.update_layout(
    title_text='Daily Average Pollutant Concentrations',
    height=300 * len(pollutants),
    hovermode='x unified'
)

# Update subplot titles font size
for annotation in fig.layout.annotations:
     if annotation.text in [f'Daily Average {p}' for p in pollutants]:
        annotation.font.size = 14

fig.show()

fig.write_image('Plotly_Plots/EDA_plots/time_series_pollutants_plotly.png', scale=2)


In [ ]:
# Ensure df_clean is defined, has DateTime index, and pd, go are imported
# df_clean = airquality.copy()
# df_clean['DateTime'] = pd.to_datetime(df_clean['Date'].astype(str) + ' ' + df_clean['Time'].astype(str))
# df_clean = df_clean.set_index('DateTime') # Ensure this is done before this cell

# --- Plotly Environmental Variable Time Series ---
env_vars = ['T', 'RH', 'AH']

# Resample data first
df_env_daily_mean = df_clean[env_vars].resample('D').mean()

# Create subplots
fig = make_subplots(rows=len(env_vars), cols=1, shared_xaxes=True,
                    subplot_titles=[f'Daily Average {v}' for v in env_vars])

for i, var in enumerate(env_vars):
    if var == 'T': ylabel = 'Temperature (°C)'
    elif var == 'RH': ylabel = 'Relative Humidity (%)'
    elif var == 'AH': ylabel = 'Absolute Humidity (g/m³)'
    else: ylabel = 'Value'
    
    fig.add_trace(
        go.Scatter(
            x=df_env_daily_mean.index,
            y=df_env_daily_mean[var],
            mode='lines',
            name=var,
            showlegend=False,
            hovertemplate=f'<b>{var}</b><br>Date: %{{x|%Y-%m-%d}}<br>{ylabel}: %{{y:.2f}}<extra></extra>'
        ),
        row=i + 1, col=1
    )
    fig.update_yaxes(title_text=ylabel, row=i + 1, col=1)

fig.update_layout(
    title_text='Daily Average Environmental Variables',
    height=300 * len(env_vars),
    hovermode='x unified'
)

# Update subplot titles font size
for annotation in fig.layout.annotations:
    if annotation.text in [f'Daily Average {v}' for v in env_vars]: # Check if it's a subplot title
        annotation.font.size = 14

fig.show()

fig.write_image('Plotly_Plots/EDA_plots/time_series_env_plotly.png', scale=2)


In [ ]:
# Ensure df_clean, pd, np, go, make_subplots are defined/imported
# df_clean = airquality.copy() # If needed

# --- Plotly Pair Plot Approximation ---
key_vars = ['CO(GT)', 'NOx(GT)', 'NO2(GT)', 'T', 'RH']
# Sample data and drop NA for plotting
df_sample = df_clean[key_vars].dropna().sample(n=min(1000, len(df_clean.dropna())), random_state=42) # Sample safely

n_vars = len(key_vars)
fig = make_subplots(rows=n_vars, cols=n_vars, 
                    # shared_xaxes=True, shared_yaxes=True, # Causes issues with histograms on diag
                    horizontal_spacing=0.03, vertical_spacing=0.03)

for i in range(n_vars):
    for j in range(n_vars):
        row = i + 1
        col = j + 1
        x_var = key_vars[j]
        y_var = key_vars[i]

        if i == j: # Diagonal: Histogram
            fig.add_trace(
                go.Histogram(
                    x=df_sample[x_var],
                    name=x_var,
                    showlegend=False,
                    nbinsx=30,
                    marker_color='grey',
                    hovertemplate=f'<b>{x_var}</b><br>Range: %{{x}}<br>Count: %{{y}}<extra></extra>'
                ),
                row=row, col=col
            )
        else: # Off-diagonal: Scatter plot
            fig.add_trace(
                go.Scatter(
                    x=df_sample[x_var],
                    y=df_sample[y_var],
                    mode='markers',
                    name='',
                    showlegend=False,
                    marker=dict(size=3, opacity=0.6, color='blue'),
                    hovertemplate=f'<b>{x_var}</b>: %{{x:.2f}}<br><b>{y_var}</b>: %{{y:.2f}}<extra></extra>'
                ),
                row=row, col=col
            )

        # Add axis labels only to the bottom row and leftmost column
        if i == n_vars - 1:
            fig.update_xaxes(title_text=x_var, row=row, col=col, title_font=dict(size=10), tickfont=dict(size=8))
        else:
             fig.update_xaxes(showticklabels=False, row=row, col=col) # Hide ticks for inner plots

        if j == 0:
            fig.update_yaxes(title_text=y_var, row=row, col=col, title_font=dict(size=10), tickfont=dict(size=8))
        else:
             fig.update_yaxes(showticklabels=False, row=row, col=col) # Hide ticks for inner plots

fig.update_layout(
    title_text='Pair Plot Approximation of Key Variables (Sampled Data)',
    height=800, width=800,
    hovermode='closest',
    bargap=0.01, # For histograms
    showlegend=False # Ensure no legend items appear
)

fig.show()

fig.write_image('Plotly_Plots/EDA_plots/pairplot_plotly.png', scale=1.5)


## EDA Summary: Key Findings and Observations

### Data Description and Patterns
- The dataset contains 9,357 hourly records of air quality and meteorological variables from an Italian city.
- Key variables include concentrations of CO, NOx, NO2, C6H6, and sensor responses, as well as temperature (T), relative humidity (RH), and absolute humidity (AH).
- There are significant missing values in some variables, especially NMHC(GT) (~90% missing), and moderate missingness in CO(GT), NOx(GT), and NO2(GT) (~18%).
- The summary statistics show a wide range of values for pollutants, with some variables (e.g., CO(GT), NOx(GT)) having outliers and skewed distributions.

### Visual Patterns and Anomalies
- Histograms reveal that many pollutant concentrations are right-skewed, with a majority of values clustered at the lower end and a long tail of higher values.
- Box plots confirm the presence of outliers, especially for CO(GT), NOx(GT), and C6H6(GT).
- Time series plots show clear daily and seasonal trends in pollutant concentrations and meteorological variables. For example, CO and NOx levels tend to be higher in colder months.
- Pair plots (scatter plots) indicate positive correlations between some pollutants (e.g., CO and NOx), and relationships between temperature/humidity and pollutant levels.

### Interesting Observations
- The high proportion of missing data in NMHC(GT) may require imputation or exclusion from some analyses.
- Outliers and non-normal distributions suggest the need for robust statistical methods or data transformation in further modeling.
- The data's temporal structure (hourly, with date and time) enables time series analysis and investigation of diurnal/seasonal cycles.

### Next Steps
- Address missing values and outliers in preprocessing.
- Explore feature engineering and correlation analysis for predictive modeling.
- Consider stratified or time-based data splitting for model validation.

In [ ]:
print("Exploratory data analysis completed. Visualizations saved to exploratory_data_analysis/ directory")

## Phase 2 Summary

The exploratory data analysis phase provided valuable insights into the distribution, patterns, and relationships within the air quality dataset. Through various visualizations and statistical analyses, we gained a deeper understanding of the data characteristics and identified important features for further investigation.

Key findings from this phase include:

- Pollutant concentrations (CO, NOx, NO2, C6H6) showed right-skewed distributions, with most values clustered at the lower end and a long tail of higher values.
- Temperature (T) followed a bimodal distribution, reflecting seasonal variations throughout the year.
- Clear temporal patterns were observed in pollution levels, with distinct daily, weekly, and seasonal variations.
- Strong correlations were found between related pollutants (e.g., NOx and NO2), and between pollutants and their corresponding sensor readings.
- Environmental factors like temperature and humidity showed significant relationships with pollution levels, with lower temperatures often associated with higher pollution concentrations.

These insights inform our approach to data preprocessing, feature engineering, and modeling in subsequent phases. The identified patterns and relationships will guide our selection of relevant features and appropriate modeling techniques.

# Phase 3: Data Preprocessing

### 3.1 Data Cleaning

Handling missing values, removing outliers, and preparing data for analysis.

In [ ]:
print("\n--- Phase 3.1: Initial Overview, Duplicates, and Missing Values ---")

In [ ]:
# Initial overview
print("Initial Dataset Overview:")
print(f"Number of observations: {airquality.shape[0]}")
print(f"Number of variables: {airquality.shape[1]}")

In [ ]:
# Check and remove duplicates
duplicates = airquality.duplicated().sum()
print(f"\nDuplicate Records: {duplicates}")
if duplicates > 0:
    print("Removing duplicate records...")
    airquality = airquality.drop_duplicates()
    print(f"Dataset shape after removing duplicates: {airquality.shape}")
else:
    print("No duplicate records found.")

### 3.2 Feature Engineering

Creating new features to enhance analysis and modeling capabilities.

In [ ]:
# Handle -200 as missing values
airquality_clean = airquality.copy()
print("\nMissing Values Before Treatment:")
for col in airquality_clean.columns:
    if airquality_clean[col].dtype != 'datetime64[ns]' and airquality_clean[col].dtype != 'object':
        mask = airquality_clean[col] == -200
        missing_count = mask.sum()
        if missing_count > 0:
            print(f"{col}: {missing_count} missing values ({missing_count/len(airquality_clean)*100:.2f}%)")
            airquality_clean.loc[mask, col] = np.nan

print("\nMissing Values Treatment Strategy:")
for col in airquality_clean.columns:
    if col not in ['Date', 'Time'] and airquality_clean[col].isna().sum() > 0:
        missing_pct = airquality_clean[col].isna().sum() / len(airquality_clean) * 100
        if missing_pct > 80:
            print(f"{col}: {missing_pct:.2f}% missing - Column will be dropped")
        elif missing_pct > 30:
            print(f"{col}: {missing_pct:.2f}% missing - Sensor correlations will be used for imputation")
        else:
            print(f"{col}: {missing_pct:.2f}% missing - Forward fill with rolling mean")

# Drop high-missing column
if 'NMHC(GT)' in airquality_clean.columns and airquality_clean['NMHC(GT)'].isna().sum() / len(airquality_clean) > 0.8:
    print("Dropping NMHC(GT) due to excessive missing values")
    airquality_clean = airquality_clean.drop(columns=['NMHC(GT)'])

In [ ]:
print("\n--- Phase 3.2: Imputation and Outlier Handling ---")

In [ ]:
# Set datetime index
airquality_clean['DateTime'] = pd.to_datetime(airquality_clean['Date'].astype(str) + ' ' + airquality_clean['Time'].astype(str))
airquality_clean = airquality_clean.set_index('DateTime').sort_index()

In [ ]:
# Sensor-based imputation
pollutant_sensor_pairs = [('CO(GT)', 'PT08.S1(CO)'), ('NOx(GT)', 'PT08.S3(NOx)'), ('NO2(GT)', 'PT08.S4(NO2)')]
for pollutant, sensor in pollutant_sensor_pairs:
    if pollutant in airquality_clean.columns and sensor in airquality_clean.columns:
        if airquality_clean[pollutant].isna().sum() > 0:
            valid_data = airquality_clean[[pollutant, sensor]].dropna()
            if len(valid_data) > 0:
                correlation = valid_data[pollutant].corr(valid_data[sensor])
                print(f"Correlation between {pollutant} and {sensor}: {correlation:.4f}")
                if abs(correlation) > 0.5:
                    model = LinearRegression()
                    model.fit(valid_data[[sensor]], valid_data[pollutant])
                    predict_indices = airquality_clean[pollutant].isna() & ~airquality_clean[sensor].isna()
                    airquality_clean.loc[predict_indices, pollutant] = model.predict(airquality_clean.loc[predict_indices, [sensor]])
                    print(f"Used regression model to impute {predict_indices.sum()} values in {pollutant}")

### 3.3 Temporal Feature Engineering

Extracting time-based features from datetime information.

In [ ]:
# Rolling mean and fill
for col in airquality_clean.columns:
    if col not in ['Date', 'Time'] and airquality_clean[col].isna().sum() > 0:
        missing_before = airquality_clean[col].isna().sum()
        rolling_mean = airquality_clean[col].rolling(window=24, min_periods=1).mean()
        airquality_clean[col] = airquality_clean[col].fillna(rolling_mean)
        if airquality_clean[col].isna().sum() > 0:
            airquality_clean[col] = airquality_clean[col].ffill()
        if airquality_clean[col].isna().sum() > 0:
            airquality_clean[col] = airquality_clean[col].bfill()
        if airquality_clean[col].isna().sum() > 0:
            airquality_clean[col] = airquality_clean[col].fillna(airquality_clean[col].mean())
        print(f"{col}: Imputed {missing_before} missing values")

print(f"\nRemaining missing values: {airquality_clean.isna().sum().sum()}")

In [ ]:
# Outlier handling
numeric_cols = airquality_clean.select_dtypes(include=['float64', 'int64']).columns
numeric_cols = [col for col in numeric_cols if col not in ['Date', 'Time']]

In [ ]:
# Create a melted dataframe for visualization
df_melted = pd.melt(airquality_clean.reset_index()[numeric_cols], var_name='variable', value_name='value')

# Create boxplot
fig = px.box(
    df_melted,
    x='variable',
    y='value',
    title='Box Plots Before Outlier Treatment',
    labels={'variable': 'Variables', 'value': 'Values'},
    height=600
)

# Rotate x-axis labels for better readability
fig.update_layout(
    xaxis_tickangle=-45,
    xaxis_title=None,
    yaxis_title='Values',
    margin=dict(l=50, r=50, t=80, b=100),
    template='plotly_white'  # Clean white background
)

# Show plot
fig.show()

fig.write_image('Plotly_Plots/preprocessing_plots/before_outlier_treatment_boxplot_plotly.png', scale=1.5)




In [ ]:
# Detect and cap outliers
for col in numeric_cols:
    Q1 = airquality_clean[col].quantile(0.25)
    Q3 = airquality_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = ((airquality_clean[col] < lower) | (airquality_clean[col] > upper)).sum()
    print(f"{col}: {outliers} outliers detected")
    if outliers > 0:
        airquality_clean[col] = airquality_clean[col].clip(lower, upper)
        print(f"  - Outliers capped between {lower:.2f} and {upper:.2f}")

In [ ]:

# Create a melted dataframe for visualization
df_melted = pd.melt(airquality_clean.reset_index()[numeric_cols], var_name='variable', value_name='value')

# Create boxplot
fig = px.box(
    df_melted,
    x='variable',
    y='value',
    title='Box Plots Before Outlier Treatment',
    labels={'variable': 'Variables', 'value': 'Values'},
    height=600
)

# Rotate x-axis labels for better readability
fig.update_layout(
    xaxis_tickangle=-45,
    xaxis_title=None,
    yaxis_title='Values',
    margin=dict(l=50, r=50, t=80, b=100),
    template='plotly_white'  # Clean white background
)

# Show plot
fig.show()

fig.write_image('Plotly_Plots/preprocessing_plots/after_outlier_treatment_boxplot_plotly.png', scale=1.5)


In [ ]:
print("\n--- Phase 3.3: Data Transformation and Finalization ---")

In [ ]:
# Standardization
print("Standardizing numeric features...")
airquality_standardized = airquality_clean.copy()
scaler = StandardScaler()
for col in numeric_cols:
    airquality_standardized[col] = scaler.fit_transform(airquality_standardized[[col]])

In [ ]:
# Save preprocessed and standardized data
airquality_clean.to_csv('preprocessing/preprocessed_data.csv')
airquality_standardized.to_csv('preprocessing/standardized_data.csv')

print(f"\nFinal Preprocessed Dataset Shape: {airquality_clean.shape}")
print(f"Columns: {list(airquality_clean.columns)}")
print("Preprocessed data saved to 'preprocessing/preprocessed_data.csv'")
print("Standardized data saved to 'preprocessing/standardized_data.csv'")

In [ ]:
print("Data preprocessing completed. Results saved to preprocessing/ directory")

# Phase 3.4: Feature Engineering & Advanced Preprocessing

In this section, we will create new features to enhance our analysis and modeling capabilities. These engineered features will help capture temporal patterns, environmental conditions, and other factors that may influence air quality.

In [ ]:
print("\n--- Phase 3.4: Feature Engineering & Advanced Preprocessing ---")

# Load the preprocessed data
df = pd.read_csv('preprocessing/preprocessed_data.csv', index_col=0, parse_dates=True)
df.head()

## Rush Hour Indicator

Creating a feature to indicate if a timestamp falls within typical rush hours (7-9 AM, 5-8 PM).

In [ ]:
# Extract hour from the datetime index
df['hour'] = df.index.hour

# Create rush hour indicator
df['is_rush_hour'] = ((df['hour'] >= 7) & (df['hour'] <= 9)) | ((df['hour'] >= 17) & (df['hour'] <= 20))
df['is_rush_hour'] = df['is_rush_hour'].astype(int)  # Convert boolean to 0/1

# Visualize pollution levels during rush hours vs. non-rush hours using plotly
pollutants = ['CO(GT)', 'NOx(GT)', 'NO2(GT)']
fig = make_subplots(rows=len(pollutants), cols=1, 
                    subplot_titles=[f'{p} Levels: Rush Hour vs. Non-Rush Hour' for p in pollutants],
                    vertical_spacing=0.1)

for i, pollutant in enumerate(pollutants):
    # Create box plots for rush hour (1)
    fig.add_trace(
        go.Box(
            x=df[df['is_rush_hour']==1]['is_rush_hour'],
            y=df[df['is_rush_hour']==1][pollutant],
            name="Rush Hour",
            boxmean=True,  # adds a marker for the mean
            marker_color='darkblue',
            hovertemplate=f"{pollutant}: %{{y:.2f}}<extra></extra>"
        ),
        row=i+1, col=1
    )
    
    # Create box plots for non-rush hour (0)
    fig.add_trace(
        go.Box(
            x=df[df['is_rush_hour']==0]['is_rush_hour'],
            y=df[df['is_rush_hour']==0][pollutant],
            name="Non-Rush Hour",
            boxmean=True,
            marker_color='lightblue',
            hovertemplate=f"{pollutant}: %{{y:.2f}}<extra></extra>"
        ),
        row=i+1, col=1
    )
    
    fig.update_xaxes(
        ticktext=["Non-Rush Hour", "Rush Hour"], 
        tickvals=[0, 1],
        title_text="Time Period",
        row=i+1, col=1
    )
    
    fig.update_yaxes(
        title_text="Concentration",
        row=i+1, col=1
    )

fig.update_layout(
    height=800,
    width=800,
    title_text="Pollution Levels: Rush Hour vs. Non-Rush Hour",
    showlegend=i==0  # Only show legend for the first pollutant
)

fig.show()

fig.write_image('Plotly_Plots/preprocessing_plots/rush_hour_boxplot_plotly.png', scale=1.5)

### Justification for Rush Hour Indicator

The rush hour indicator is valuable for several reasons:

1. **Traffic Patterns**: Rush hours typically coincide with increased traffic volume, which is a major source of urban air pollution.
2. **Predictive Power**: This feature can help models identify and predict pollution spikes associated with commuting patterns.
3. **Policy Relevance**: Understanding pollution patterns during rush hours can inform traffic management policies and public health advisories.
4. **Temporal Context**: It provides important temporal context that raw timestamp data doesn't explicitly capture.

## Weekend vs. Weekday Feature

Creating a binary feature to distinguish between weekdays and weekends.

In [ ]:
# Extract day of week (0=Monday, 6=Sunday)
df['day_of_week'] = df.index.dayofweek

# Create weekend indicator (5=Saturday, 6=Sunday)
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

# Visualize pollution levels on weekends vs. weekdays using plotly
fig = make_subplots(rows=3, cols=1, 
                    subplot_titles=[f'{pollutant} Levels: Weekend vs. Weekday' for pollutant in ['CO(GT)', 'NOx(GT)', 'NO2(GT)']])

for i, pollutant in enumerate(['CO(GT)', 'NOx(GT)', 'NO2(GT)']):
    # Create box plots for weekend (1)
    fig.add_trace(
        go.Box(
            x=df[df['is_weekend']==1]['is_weekend'],
            y=df[df['is_weekend']==1][pollutant],
            name="Weekend",
            boxmean=True,
            marker_color='darkgreen',
            hovertemplate=f"{pollutant}: %{{y:.2f}}<extra></extra>"
        ),
        row=i+1, col=1
    )
    
    # Create box plots for weekday (0)
    fig.add_trace(
        go.Box(
            x=df[df['is_weekend']==0]['is_weekend'],
            y=df[df['is_weekend']==0][pollutant],
            name="Weekday",
            boxmean=True,
            marker_color='lightgreen',
            hovertemplate=f"{pollutant}: %{{y:.2f}}<extra></extra>"
        ),
        row=i+1, col=1
    )
    
    fig.update_xaxes(
        ticktext=["Weekday", "Weekend"], 
        tickvals=[0, 1],
        title_text="Day Type",
        row=i+1, col=1
    )
    
    fig.update_yaxes(
        title_text="Concentration",
        row=i+1, col=1
    )

fig.update_layout(
    height=800,
    width=800,
    title_text="Pollution Levels: Weekend vs. Weekday",
    showlegend=i==0  # Only show legend for the first subplot
)

fig.show()

fig.write_image('Plotly_Plots/preprocessing_plots/weekend_boxplot_plotly.png', scale=1.5)


### Justification for Weekend vs. Weekday Feature

The weekend/weekday distinction is important for these reasons:

1. **Activity Patterns**: Human activity patterns differ significantly between weekdays and weekends, affecting emission sources.
2. **Industrial Operations**: Many industrial facilities operate on different schedules during weekends.
3. **Traffic Volumes**: Traffic patterns and volumes typically differ between weekdays and weekends.
4. **Model Accuracy**: Including this feature can help models account for weekly cyclical patterns in pollution levels.

## Season Feature

Creating a categorical feature to represent seasons, which can significantly affect pollution patterns.

In [ ]:
# Extract month
df['month'] = df.index.month

# Create season feature (Northern Hemisphere)
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:  # 9, 10, 11
        return 'Fall'

df['season'] = df['month'].apply(get_season)

# Visualize pollution levels by season
pollutants = ['CO(GT)', 'NOx(GT)', 'NO2(GT)']
fig = go.Figure()

# Create subplots - one row per pollutant
fig = make_subplots(
    rows=len(pollutants),
    cols=1,
    subplot_titles=[f'{p} Levels by Season' for p in pollutants],
    vertical_spacing=0.1
)

seasons = ['Winter', 'Spring', 'Summer', 'Fall']
colors = ['#A1D6E2', '#81B29A', '#F2CC8F', '#E07A5F']

for i, pollutant in enumerate(pollutants):
    for j, season in enumerate(seasons):
        fig.add_trace(
            go.Box(
                y=df[df['season'] == season][pollutant],
                name=season,
                marker_color=colors[j],
                showlegend=i==0,  # Only show legend for the first pollutant
                hovertemplate=f"{pollutant} ({season}): %{{y:.2f}}<extra></extra>"
            ),
            row=i+1, col=1
        )
    
    fig.update_yaxes(title_text='Concentration', row=i+1, col=1)

fig.update_layout(
    height=800,
    boxmode='group',
    title_text='Pollution Levels by Season',
    legend_title_text='Season'
)

fig.show()

fig.write_image('Plotly_Plots/preprocessing_plots/season_boxplot_plotly.png', scale=1.5)


### Justification for Season Feature

The season feature is valuable for these reasons:

1. **Meteorological Conditions**: Seasons bring different weather patterns that affect pollution dispersion and chemistry.
2. **Emission Sources**: Seasonal activities like heating in winter or increased air conditioning in summer affect emissions.
3. **Photochemical Reactions**: Seasonal variations in sunlight affect photochemical reactions that create secondary pollutants.
4. **Long-term Patterns**: This feature helps models capture long-term cyclical patterns in the data.

## Time-Based Features

Creating additional time-based features to capture temporal patterns at different scales.

In [ ]:
# Hour of day as cyclical features using sine and cosine transformations
# This preserves the cyclical nature of time (hour 23 is close to hour 0)
df['hour_sin'] = np.sin(2 * np.pi * df['hour']/24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour']/24)

# Day of week as cyclical features
df['day_sin'] = np.sin(2 * np.pi * df['day_of_week']/7)
df['day_cos'] = np.cos(2 * np.pi * df['day_of_week']/7)

# Month as cyclical features
df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
df['month_cos'] = np.cos(2 * np.pi * df['month']/12)

# Create subplots with plotly
fig = make_subplots(rows=2, cols=1, 
                    subplot_titles=["Average CO(GT) by Hour of Day", 
                                   "Cyclical Representation of Hours"])

# Average CO by hour plot
hourly_co = df.groupby('hour')['CO(GT)'].mean()
fig.add_trace(
    go.Scatter(
        x=hourly_co.index, 
        y=hourly_co.values,
        mode='lines+markers',
        name='CO(GT)',
        line=dict(color='blue'),
        hovertemplate='Hour: %{x}<br>CO(GT): %{y:.2f}<extra></extra>'
    ),
    row=1, col=1
)

# Cyclical representation plot
fig.add_trace(
    go.Scatter(
        x=df['hour_sin'],
        y=df['hour_cos'],
        mode='markers',
        marker=dict(
            size=8,
            color=df['hour'],
            colorscale='Viridis',
            colorbar=dict(
                title='Hour of Day',
                x=1,
                y=0.5,
                yanchor='middle'
            ),
            showscale=True
        ),
        hovertemplate='Hour: %{marker.color:.0f}<br>sin(hour): %{x:.2f}<br>cos(hour): %{y:.2f}<extra></extra>'
    ),
    row=2, col=1
)

# Update layout
fig.update_layout(
    height=800,
    width=900,
    title_text='Cyclical Time Features Representation',
    showlegend=False,
    hovermode='closest'
)

# Update axes
fig.update_xaxes(title_text='Hour of Day', row=1, col=1)
fig.update_yaxes(title_text='CO(GT) Concentration', row=1, col=1)
fig.update_xaxes(title_text='sin(hour)', row=2, col=1)
fig.update_yaxes(title_text='cos(hour)', row=2, col=1)

fig.show()

fig.write_image('Plotly_Plots/preprocessing_plots/cyclical_time_features_plotly.png', scale=1.5)


### Justification for Cyclical Time Features

Cyclical time features offer several advantages:

1. **Preserving Cyclical Nature**: They maintain the cyclical relationship between time periods (e.g., hour 23 is close to hour 0).
2. **Model Compatibility**: These transformations make time features more suitable for machine learning models.
3. **Capturing Periodicity**: They help models identify periodic patterns at different time scales (daily, weekly, monthly).
4. **Feature Importance**: They often provide more predictive power than raw time values.

## Lag Features

Creating lag features to capture the relationship between current and past pollution levels.

In [ ]:
from plotly.subplots import make_subplots

# Create lag features for key pollutants
for pollutant in ['CO(GT)', 'NOx(GT)', 'NO2(GT)']:
    # 1-hour lag
    df[f'{pollutant}_lag1'] = df[pollutant].shift(1)
    # 24-hour lag (same time yesterday)
    df[f'{pollutant}_lag24'] = df[pollutant].shift(24)
    # 168-hour lag (same time last week)
    df[f'{pollutant}_lag168'] = df[pollutant].shift(168)

# Create rolling average features
for pollutant in ['CO(GT)', 'NOx(GT)', 'NO2(GT)']:
    # 24-hour rolling average
    df[f'{pollutant}_rolling24'] = df[pollutant].rolling(window=24).mean()
    # 7-day rolling average
    df[f'{pollutant}_rolling168'] = df[pollutant].rolling(window=168).mean()

# Drop NaN values created by lag and rolling features
df_lag = df.dropna()

# Visualize correlation between current and lagged values using plotly
import plotly.graph_objects as go

fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=['CO(GT) vs CO(GT)_lag1', 'CO(GT) vs CO(GT)_lag24', 'CO(GT) vs CO(GT)_lag168'],
    horizontal_spacing=0.1
)

lag_types = ['lag1', 'lag24', 'lag168']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

for i, lag in enumerate(lag_types):
    corr = df_lag['CO(GT)'].corr(df_lag[f'CO(GT)_{lag}']).round(3)
    
    fig.add_trace(
        go.Scatter(
            x=df_lag['CO(GT)'],
            y=df_lag[f'CO(GT)_{lag}'],
            mode='markers',
            marker=dict(
                color=colors[i],
                opacity=0.5,
                size=5
            ),
            name=f'Lag {lag.replace("lag", "")}',
            hovertemplate='Current: %{x:.2f}<br>Lag: %{y:.2f}<extra></extra>'
        ),
        row=1, col=i+1
    )
    
    # Add correlation annotation
    fig.add_annotation(
        x=0.05,
        y=0.95,
        text=f'r = {corr}',
        xref=f'x{i+1}',
        yref=f'y{i+1}',
        showarrow=False,
        font=dict(size=12),
        bgcolor='white',
        bordercolor='black',
        borderwidth=1,
        borderpad=3
    )

fig.update_layout(
    height=400,
    width=900,
    title_text='Correlation Between Current and Lagged CO(GT) Values',
    showlegend=True
)

# Update axis labels
for i in range(3):
    fig.update_xaxes(title_text='CO(GT) Current', row=1, col=i+1)
    fig.update_yaxes(title_text=f'CO(GT) {lag_types[i]}', row=1, col=i+1)

fig.show()

fig.write_image('Plotly_Plots/preprocessing_plots/correlation_plotly.png', scale=1.5)



### Justification for Lag and Rolling Features

Lag and rolling features are particularly valuable for time series data:

1. **Temporal Dependency**: They capture the autocorrelation in pollution levels over time.
2. **Predictive Power**: Previous pollution levels are often strong predictors of current levels.
3. **Trend Capture**: Rolling averages help capture longer-term trends and smooth out noise.
4. **Cyclical Patterns**: Lag features at specific intervals (24 hours, 168 hours) capture daily and weekly patterns.

## Interaction Features

Creating interaction features to capture relationships between different variables.

In [ ]:
# Create interaction features between temperature and pollutants
for pollutant in ['CO(GT)', 'NOx(GT)', 'NO2(GT)']:
    df[f'{pollutant}_T_interaction'] = df[pollutant] * df['T']

# Create interaction features between humidity and pollutants
for pollutant in ['CO(GT)', 'NOx(GT)', 'NO2(GT)']:
    df[f'{pollutant}_RH_interaction'] = df[pollutant] * df['RH']

# Create interaction between rush hour and weekend
df['rush_hour_weekend'] = df['is_rush_hour'] * df['is_weekend']

# Visualize one of the interaction features using plotly
import plotly.express as px

fig = px.scatter(
    df, 
    x='T', 
    y='CO(GT)', 
    color='CO(GT)_T_interaction',
    color_continuous_scale='viridis',
    opacity=0.6,
    title='Temperature vs CO(GT) Colored by Their Interaction',
    labels={'T': 'Temperature', 'CO(GT)': 'CO(GT) Concentration', 'CO(GT)_T_interaction': 'CO(GT) × Temperature Interaction'},
    height=600,
    width=800
)

fig.update_layout(
    coloraxis_colorbar=dict(title='CO(GT) × T'),
    hovermode='closest',
    template='plotly_white'
)

fig.show()

fig.write_image('Plotly_Plots/preprocessing_plots/interaction_plotly.png', scale=1.5)


### Justification for Interaction Features

Interaction features capture complex relationships between variables:

1. **Non-linear Relationships**: They help models capture non-linear relationships between variables.
2. **Environmental Chemistry**: Pollutant behavior often depends on interactions with environmental factors like temperature and humidity.
3. **Compound Effects**: Some effects only manifest when multiple factors coincide (e.g., rush hour on weekdays vs. weekends).
4. **Model Flexibility**: They give linear models the ability to capture more complex patterns.

In [ ]:
# Save the dataset with engineered features
df.to_csv('feature_engineering/data_with_engineered_features.csv')

# Create a summary of all engineered features
engineered_features = [
    'is_rush_hour', 'is_weekend', 'season', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'month_sin', 'month_cos',
    'CO(GT)_lag1', 'CO(GT)_lag24', 'CO(GT)_lag168', 'CO(GT)_rolling24', 'CO(GT)_rolling168',
    'NOx(GT)_lag1', 'NOx(GT)_lag24', 'NOx(GT)_lag168', 'NOx(GT)_rolling24', 'NOx(GT)_rolling168',
    'NO2(GT)_lag1', 'NO2(GT)_lag24', 'NO2(GT)_lag168', 'NO2(GT)_rolling24', 'NO2(GT)_rolling168',
    'CO(GT)_T_interaction', 'NOx(GT)_T_interaction', 'NO2(GT)_T_interaction',
    'CO(GT)_RH_interaction', 'NOx(GT)_RH_interaction', 'NO2(GT)_RH_interaction',
    'rush_hour_weekend'
]

print("\nEngineered Features Summary:")
print(f"Number of original features: {len(airquality_clean.columns)}")
print(f"Number of engineered features: {len(engineered_features)}")
print(f"Total number of features: {len(df.columns)}")
print("\nEngineered features saved to 'feature_engineering/data_with_engineered_features.csv'")

## Feature Engineering Summary

We have created several categories of engineered features to enhance our analysis and modeling capabilities:

1. **Temporal Features**:
   - Rush hour indicator
   - Weekend/weekday indicator
   - Season categorization
   - Cyclical time representations (hour, day, month)

2. **Lag and Rolling Features**:
   - 1-hour, 24-hour, and 168-hour lags for key pollutants
   - 24-hour and 7-day rolling averages

3. **Interaction Features**:
   - Pollutant × Temperature interactions
   - Pollutant × Humidity interactions
   - Rush hour × Weekend interaction

These engineered features capture important temporal patterns, environmental relationships, and complex interactions that can significantly improve model performance and provide deeper insights into air quality dynamics.

## Phase 3 Summary

In the data preprocessing and feature engineering phase, we prepared the dataset for advanced analysis and modeling by addressing data quality issues and creating new features to enhance predictive power. This phase built upon the insights gained from the exploratory data analysis to develop a clean, feature-rich dataset for subsequent modeling.

Key accomplishments in this phase include:

- Handling missing values using appropriate imputation techniques based on the nature and extent of missingness in each variable.
- Identifying and treating outliers to minimize their impact on analysis results.
- Creating temporal features (hour of day, day of week, month, season) to capture cyclical patterns in pollution levels.
- Developing lag features and rolling averages to incorporate temporal dependencies in the data.
- Normalizing and scaling variables to ensure comparability and improve model performance.

The resulting preprocessed dataset provides a solid foundation for the time series analysis and classification modeling in the following phases. The engineered features capture important temporal patterns and relationships that will enhance our ability to understand and predict air quality variations.

# Phase 4: Correlation Analysis

In this section, we will:
- Calculate and visualize the correlation matrix for all numeric variables.
- Identify and discuss significant correlations (strong positive/negative).
- Visualize key relationships with scatter plots.
- Analyze correlations between pollutants, environmental factors, and sensor performance.

In [ ]:
# Load the preprocessed data
df = pd.read_csv('preprocessing/preprocessed_data.csv', index_col=0, parse_dates=True)
df.head()

### Correlation Matrix Calculation
We calculate the correlation matrix for all numeric variables in the dataset.

In [ ]:
# Select only numeric columns
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
corr_matrix = df[numeric_cols].corr()

# Save correlation matrix to CSV
corr_matrix.to_csv('correlation_analysis/correlation_matrix.csv')

In [ ]:
corr_matrix

### Correlation Matrix Heatmap
Visualize the correlation matrix as a heatmap to better understand relationships between variables.

In [ ]:
import numpy as np

# Create heatmap visualization of correlation matrix using plotly
import plotly.express as px

# Create the heatmap with plotly
fig = px.imshow(
    corr_matrix,
    color_continuous_scale='RdBu_r',
    zmin=-1,
    zmax=1,
    text_auto='.2f',
    aspect='auto',
    title='Correlation Matrix Heatmap'
)

# Update layout for better appearance
fig.update_layout(
    width=900,
    height=800,
    xaxis_title=None,
    yaxis_title=None,
    coloraxis_colorbar=dict(
        title='Correlation',
        thicknessmode="pixels", thickness=20,
        lenmode="pixels", len=300,
        ticks="outside"
    )
)

# Show the plot
fig.show()

fig.write_image('Plotly_Plots/correlation_analysis_plots/correlation_plotly.png', scale=1.5)


### Significant Correlations
Identify strong positive (r > 0.7) and strong negative (r < -0.7) correlations.

In [ ]:
# Get upper triangle of correlation matrix to avoid duplicates
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find strong positive correlations
strong_pos = [(i, j, corr_matrix.loc[i, j]) for i in corr_matrix.index for j in corr_matrix.columns 
              if corr_matrix.loc[i, j] > 0.7 and i != j]
strong_pos.sort(key=lambda x: x[2], reverse=True)

# Find strong negative correlations
strong_neg = [(i, j, corr_matrix.loc[i, j]) for i in corr_matrix.index for j in corr_matrix.columns 
              if corr_matrix.loc[i, j] < -0.7 and i != j]
strong_neg.sort(key=lambda x: x[2])

print('Strong Positive Correlations (r > 0.7):')
if strong_pos:
    for i, j, corr in strong_pos:
        print(f'{i} and {j}: r = {corr:.4f}')
else:
    print('No strong positive correlations found (r > 0.7)')

print('\nStrong Negative Correlations (r < -0.7):')
if strong_neg:
    for i, j, corr in strong_neg:
        print(f'{i} and {j}: r = {corr:.4f}')
else:
    print('No strong negative correlations found (r < -0.7)')

### Scatter Plots for Top Correlations
Visualize the strongest positive and negative correlations with scatter plots.

In [ ]:
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import math
import os

# Create directory if it doesn't exist
os.makedirs('Plotly_Plots/correlation_analysis_plots', exist_ok=True)

# Sort all strong correlations by absolute value
all_strong = strong_pos + strong_neg
all_strong.sort(key=lambda x: abs(x[2]), reverse=True)

# Get unique pairs (avoid duplicates like A->B and B->A)
seen_pairs = set()
unique_pairs = []
for var1, var2, corr in all_strong:
    pair = tuple(sorted([var1, var2]))
    if pair not in seen_pairs:
        seen_pairs.add(pair)
        unique_pairs.append((var1, var2, corr))

# Plot each correlation separately
for i, (var1, var2, corr) in enumerate(unique_pairs):
    # Create a new figure for each correlation
    fig = go.Figure()
    
    # Color based on correlation (blue for positive, red for negative)
    color = 'blue' if corr > 0 else 'red'
    
    # Add scatter plot
    fig.add_trace(
        go.Scatter(
            x=df[var1],
            y=df[var2],
            mode='markers',
            marker=dict(
                color=color,
                opacity=0.6,
                size=5
            ),
            name='Data points',
            hovertemplate=f"{var1}: %{{x:.2f}}<br>{var2}: %{{y:.2f}}<extra></extra>"
        )
    )
    
    # Add correlation trend line
    x_range = [df[var1].min(), df[var1].max()]
    
    # Calculate trend line correctly based on actual regression
    z = np.polyfit(df[var1].dropna(), df[var2].dropna(), 1)
    p = np.poly1d(z)
    y_pred = p(x_range)
    
    fig.add_trace(
        go.Scatter(
            x=x_range,
            y=y_pred,
            mode='lines',
            line=dict(color='black', width=1.5),
            name='Trend line',
            hoverinfo='skip'
        )
    )
    
    # Update layout
    fig.update_layout(
        height=600,
        width=800,
        title=f"Correlation between {var1} and {var2} (r = {corr:.4f})",
        xaxis_title=var1,
        yaxis_title=var2,
        template='plotly_white',
        hovermode='closest'
    )
    
    # Show the figure
    fig.show()
    
    # Save the figure
    fig.write_image(f'Plotly_Plots/correlation_analysis_plots/correlation_{var1}_{var2}.png', scale=1.5)

# Also create the combined plot
n_pairs = len(unique_pairs)
n_cols = 3  # Keep 3 columns
n_rows = math.ceil(n_pairs / n_cols)  # Calculate required rows

# Create subplots for all correlations
fig = make_subplots(rows=n_rows, cols=n_cols, 
                   subplot_titles=[f"{var1} vs {var2} (r = {corr:.4f})" 
                                  for var1, var2, corr in unique_pairs])

# Add scatter plots to each subplot
for i, (var1, var2, corr) in enumerate(unique_pairs):
    row = i // n_cols + 1
    col = i % n_cols + 1
    
    # Color based on correlation (blue for positive, red for negative)
    color = 'blue' if corr > 0 else 'red'
    
    fig.add_trace(
        go.Scatter(
            x=df[var1],
            y=df[var2],
            mode='markers',
            marker=dict(
                color=color,
                opacity=0.6,
                size=5
            ),
            hovertemplate=f"{var1}: %{{x:.2f}}<br>{var2}: %{{y:.2f}}<extra></extra>"
        ),
        row=row, col=col
    )
    
    # Add correlation trend line
    x_range = [df[var1].min(), df[var1].max()]
    
    # Calculate trend line correctly based on actual regression
    z = np.polyfit(df[var1].dropna(), df[var2].dropna(), 1)
    p = np.poly1d(z)
    y_pred = p(x_range)
    
    fig.add_trace(
        go.Scatter(
            x=x_range,
            y=y_pred,
            mode='lines',
            line=dict(color='black', width=1.5),
            showlegend=False,
            hoverinfo='skip'
        ),
        row=row, col=col
    )
    
    # Set axis titles
    fig.update_xaxes(title_text=var1, row=row, col=col)
    fig.update_yaxes(title_text=var2, row=row, col=col)

# Update layout
fig.update_layout(
    height=300 * n_rows,  # Adjust height based on number of rows
    width=1000,
    title_text=f"All Strong Correlations (|r| > 0.7)",
    showlegend=False,
    hovermode='closest',
    template='plotly_white'
)

fig.show()

fig.write_image('Plotly_Plots/correlation_analysis_plots/all_correlations_combined.png', scale=1.5)


### Pollutant and Environmental Factor Correlations
Analyze correlations between pollutants and environmental factors.

In [ ]:
# Pollutant and Environmental Factor Correlations using Plotly
pollutant_cols = ['CO(GT)', 'C6H6(GT)', 'NOx(GT)', 'NO2(GT)']
env_cols = ['T', 'RH', 'AH']

# Calculate the correlation
subset_corr = df[pollutant_cols + env_cols].corr()

# Create heatmap with Plotly
import plotly.express as px

fig = px.imshow(
    subset_corr,
    color_continuous_scale='RdBu_r',
    zmin=-1,
    zmax=1,
    text_auto='.2f',
    aspect='auto',
    title='Correlation Between Pollutants and Environmental Factors'
)

fig.update_layout(
    width=900, 
    height=600,
    xaxis_title=None,
    yaxis_title=None,
    coloraxis_colorbar=dict(
        title='Correlation',
        thicknessmode="pixels", 
        thickness=20,
        lenmode="pixels", 
        len=300
    )
)

# Show the plot
fig.show()

fig.write_image('Plotly_Plots/correlation_analysis_plots/pollutant_env_factor_correlation_plotly.png', scale=1.5)

# Return the specific subset of correlations between pollutants and environmental factors
subset_corr.loc[pollutant_cols, env_cols]

### Sensor Performance Analysis
Analyze the correlation between ground truth pollutant measurements and corresponding sensor readings.

In [ ]:
from plotly.subplots import make_subplots

# Analyze the correlation between ground truth pollutant measurements and corresponding sensor readings
sensor_pairs = [
    ('CO(GT)', 'PT08.S1(CO)'),
    ('NOx(GT)', 'PT08.S3(NOx)'),
    ('NO2(GT)', 'PT08.S4(NO2)')
]

# Create subplots - one for each sensor pair
import plotly.graph_objects as go

fig = make_subplots(rows=len(sensor_pairs), cols=1, 
                    subplot_titles=[f'Correlation between {gt} and {sensor}' for gt, sensor in sensor_pairs],
                    vertical_spacing=0.12)

for i, (gt, sensor) in enumerate(sensor_pairs):
    corr_val = corr_matrix.loc[gt, sensor]
    
    # Add scatter plot
    fig.add_trace(
        go.Scatter(
            x=df[gt],
            y=df[sensor],
            mode='markers',
            marker=dict(
                opacity=0.5,
                color='steelblue',
                size=5
            ),
            name=f'{gt} vs {sensor}',
            hovertemplate=f'{gt}: %{{x:.2f}}<br>{sensor}: %{{y:.2f}}<extra></extra>'
        ),
        row=i+1, col=1
    )
    
    # Add correlation annotation
    fig.add_annotation(
        x=0.95,
        y=0.95,
        text=f'r = {corr_val:.4f}',
        xref=f'x{i+1}',
        yref=f'y{i+1}',
        showarrow=False,
        font=dict(size=14, color='red' if corr_val < 0 else 'green'),
        bgcolor='white',
        bordercolor='black',
        borderwidth=1,
        borderpad=4,
        align='right'
    )
    
    # Set axis titles
    fig.update_xaxes(title_text=gt, row=i+1, col=1)
    fig.update_yaxes(title_text=sensor, row=i+1, col=1)

# Update layout
fig.update_layout(
    height=900,
    width=800,
    title_text='Correlation between Ground Truth Pollutants and Sensor Readings',
    showlegend=False,
    template='plotly_white'
)

fig.show()

fig.write_image('Plotly_Plots/correlation_analysis_plots/pollutant_sensor_correlation_plotly.png', scale=1.5)



### Correlation Analysis Summary
- The strongest correlations in the dataset are highlighted above.
- Pollutant and environmental factor correlations reveal how weather conditions may influence pollution levels.
- Sensor performance analysis shows the relationship between sensor readings and ground truth measurements.
- These insights can guide feature selection and further modeling.

In [ ]:
print("Correlation analysis completed. Results saved to correlation_analysis/ directory")

# Phase 5: Time Series Analysis

This phase delves into the temporal characteristics of the air quality data. We will perform time series decomposition to identify trend, seasonality, and residuals. Stationarity tests will be conducted, followed by ACF/PACF analysis to inform ARIMA modeling. Finally, an ARIMA model will be developed for forecasting key pollutant concentrations.

### 5.1 Loading Data for Time Series Analysis

Loading the preprocessed data and selecting key pollutants for the time series analysis.

In [ ]:
df_tsa = pd.read_csv('preprocessing/preprocessed_data.csv', index_col='DateTime', parse_dates=True)
print(f"Data for Time Series Analysis loaded. Shape: {df_tsa.shape}")
print(f"Time range: {df_tsa.index.min()} to {df_tsa.index.max()}")

# Select key pollutants for time series analysis
pollutants_tsa = ['CO(GT)', 'NOx(GT)', 'NO2(GT)', 'C6H6(GT)']
# Ensure these columns exist
pollutants_tsa = [p for p in pollutants_tsa if p in df_tsa.columns]
print(f"Selected pollutants for TSA: {pollutants_tsa}")

### 5.2 Visualizing Temporal Patterns

Plotting daily and monthly average concentrations to observe overall trends and seasonal patterns.

In [ ]:
# Converted from matplotlib/seaborn to Plotly
# Resample data to daily averages for better visualization
df_daily_tsa = df_tsa[pollutants_tsa].resample('D').mean()

fig = go.Figure(data=[go.Scatter(x=df_daily_tsa.index, y=df_daily_tsa[pollutant], name=pollutant) for pollutant in pollutants_tsa])
fig.update_layout(title='Daily Average Concentrations of Key Pollutants', yaxis_title='Concentration')
fig.write_image("Plotly_Plots/time_series_analysis_plots/daily_pollutants_tsa.png", scale=1.5)
fig.show()

# Monthly averages for seasonal patterns
df_monthly_tsa = df_tsa[pollutants_tsa].resample('M').mean()

fig = go.Figure(data=[go.Scatter(x=df_monthly_tsa.index, y=df_monthly_tsa[pollutant], name=pollutant) for pollutant in pollutants_tsa])
fig.update_layout(title='Monthly Average Concentrations of Key Pollutants', yaxis_title='Concentration')
fig.write_image("Plotly_Plots/time_series_analysis_plots/monthly_pollutants_tsa.png", scale=1.5)
fig.show()
print("Daily and monthly average pollutant concentrations plotted and saved.")

Analyzing and plotting average hourly and weekly patterns for key pollutants to identify diurnal and weekly cycles.

In [ ]:
# Converted from matplotlib/seaborn to Plotly
# Hourly patterns (average by hour of day)
df_tsa_hourly = df_tsa.copy()
df_tsa_hourly['hour'] = df_tsa_hourly.index.hour
hourly_patterns_tsa = df_tsa_hourly.groupby('hour')[pollutants_tsa].mean()

fig = go.Figure()
for i, pollutant in enumerate(pollutants_tsa):
    fig.add_trace(go.Scatter(x=hourly_patterns_tsa.index, y=hourly_patterns_tsa[pollutant],
                             mode='lines', name=pollutant))
fig.update_layout(title='Average Pollutant Concentrations by Hour of Day',
                  yaxis_title='Concentration', xaxis_title='Hour of Day')
fig.update_xaxes(tickvals=list(range(0, 24, 2)))  # Changed: wrapped range() in list()
fig.write_image('time_series_analysis/hourly_patterns_tsa.png', format='png', engine='kaleido') # Requires kaleido package
fig.show()

# Weekly patterns (average by day of week)
df_tsa_weekly = df_tsa.copy()
df_tsa_weekly['day_of_week'] = df_tsa_weekly.index.dayofweek # Monday=0, Sunday=6
weekly_patterns_tsa = df_tsa_weekly.groupby('day_of_week')[pollutants_tsa].mean()
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

fig = go.Figure()
for i, pollutant in enumerate(pollutants_tsa):
    fig.add_trace(go.Bar(x=weekly_patterns_tsa.index, y=weekly_patterns_tsa[pollutant],
                         name=pollutant))
fig.update_layout(title='Average Pollutant Concentrations by Day of Week',
                  yaxis_title='Concentration', xaxis_title='Day of Week')
fig.update_xaxes(tickvals=list(range(7)), ticktext=days, tickangle=45)  # Changed: wrapped range() in list()
fig.write_image('Plotly_Plots/time_series_analysis_plots/weekly_patterns_tsa.png', format='png', engine='kaleido') # Requires kaleido package
fig.show()
print("Hourly and weekly average pollutant patterns plotted and saved.")

correct this plotly code and save the plot

### 5.3 Time Series Decomposition

Decomposing the CO(GT) time series into trend, seasonal, and residual components.

In [ ]:
# Converted from matplotlib/seaborn to Plotly
from statsmodels.tsa.seasonal import seasonal_decompose

# Select CO(GT) for detailed decomposition analysis from daily data
target_pollutant_decomp = 'CO(GT)'
if target_pollutant_decomp in df_daily_tsa.columns:
    ts_decomp = df_daily_tsa[target_pollutant_decomp].fillna(method='ffill').fillna(method='bfill')
    
    if not ts_decomp.empty and len(ts_decomp.dropna()) >= 2 * 30: # Ensure enough data for period 30
        decomposition = seasonal_decompose(ts_decomp.dropna(), model='additive', period=30)
        
        # Create subplots
        fig = make_subplots(rows=4, cols=1, 
                           subplot_titles=["Observed", "Trend", "Seasonal", "Residual"],
                           vertical_spacing=0.1)
        
        # Add traces for each component
        fig.add_trace(go.Scatter(x=decomposition.observed.index, y=decomposition.observed, 
                                mode='lines', name='Observed'), row=1, col=1)
        fig.add_trace(go.Scatter(x=decomposition.trend.index, y=decomposition.trend, 
                                mode='lines', name='Trend'), row=2, col=1)
        fig.add_trace(go.Scatter(x=decomposition.seasonal.index, y=decomposition.seasonal, 
                                mode='lines', name='Seasonal'), row=3, col=1)
        fig.add_trace(go.Scatter(x=decomposition.resid.index, y=decomposition.resid, 
                                mode='lines', name='Residual'), row=4, col=1)
        
        # Update layout
        fig.update_layout(
            height=800, 
            width=1000,
            title_text=f'Time Series Decomposition of Daily {target_pollutant_decomp}',
            showlegend=False
        )
        
        fig.write_image("Plotly_Plots/time_series_analysis_plots/decomposition_co.png", format='png', engine='kaleido')  # Requires kaleido package
        fig.show()
        print(f"Time series decomposition for {target_pollutant_decomp} completed and saved.")
    else:
        print(f"Not enough data points for decomposition of {target_pollutant_decomp} with period 30 after dropping NaNs. Length: {len(ts_decomp.dropna())}")
else:
    print(f"'{target_pollutant_decomp}' not found in daily resampled data for decomposition.")

### 5.4 Stationarity Analysis

Performing the Augmented Dickey-Fuller (ADF) test to check for stationarity in the CO(GT) time series.

In [ ]:
from statsmodels.tsa.stattools import adfuller

if target_pollutant_decomp in df_daily_tsa.columns and 'ts_decomp' in locals() and not ts_decomp.empty:
    ts_adf = ts_decomp.dropna()
    if not ts_adf.empty:
        print(f"--- Stationarity Analysis for {target_pollutant_decomp} ---")
        result_adf = adfuller(ts_adf)
        print(f'ADF Statistic: {result_adf[0]}')
        print(f'p-value: {result_adf[1]}')
        print('Critical Values:')
        for key, value in result_adf[4].items():
            print(f'  {key}: {value}')

        if result_adf[1] <= 0.05:
            print(f"Conclusion: The time series for {target_pollutant_decomp} is stationary (reject H0).")
        else:
            print(f"Conclusion: The time series for {target_pollutant_decomp} is not stationary (fail to reject H0). Differencing may be needed.")
    else:
        print(f"Time series for {target_pollutant_decomp} is empty after dropping NaNs for ADF test.")
else:
    print(f"'{target_pollutant_decomp}' or its time series 'ts_decomp' not available for ADF test.")

### 5.5 Autocorrelation and Partial Autocorrelation Analysis

Generating ACF and PACF plots for the CO(GT) time series to help determine ARIMA model parameters.

In [ ]:
# Converted from matplotlib/seaborn to Plotly
from statsmodels.tsa.stattools import acf, pacf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

if 'ts_adf' in locals() and not ts_adf.empty:
    # Calculate ACF and PACF values
    acf_values = acf(ts_adf, nlags=40)
    pacf_values = pacf(ts_adf, nlags=40)
    
    # Create x-axis values (lags)
    lags = list(range(len(acf_values)))
    
    # Create subplots
    fig = make_subplots(rows=1, cols=2, subplot_titles=[f'ACF for {target_pollutant_decomp}', 
                                                        f'PACF for {target_pollutant_decomp}'])
    
    # Add ACF plot
    fig.add_trace(
        go.Bar(x=lags, y=acf_values, name='ACF'),
        row=1, col=1
    )
    
    # Add PACF plot
    fig.add_trace(
        go.Bar(x=lags, y=pacf_values, name='PACF'),
        row=1, col=2
    )
    
    # Add confidence intervals (approximately 95% CI is ±1.96/sqrt(n))
    conf_level = 1.96 / np.sqrt(len(ts_adf))
    
    # Add confidence intervals to ACF plot
    fig.add_trace(
        go.Scatter(x=lags, y=[conf_level] * len(lags), mode='lines', line=dict(dash='dash', color='red'), 
                  showlegend=False),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=lags, y=[-conf_level] * len(lags), mode='lines', line=dict(dash='dash', color='red'), 
                  showlegend=False),
        row=1, col=1
    )
    
    # Add confidence intervals to PACF plot
    fig.add_trace(
        go.Scatter(x=lags, y=[conf_level] * len(lags), mode='lines', line=dict(dash='dash', color='red'), 
                  showlegend=False),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(x=lags, y=[-conf_level] * len(lags), mode='lines', line=dict(dash='dash', color='red'), 
                  showlegend=False),
        row=1, col=2
    )
    
    # Update layout
    fig.update_layout(
        height=500, 
        width=900,
        title_text=f'ACF and PACF for {target_pollutant_decomp}',
        showlegend=False
    )
    
    # Update axes
    fig.update_xaxes(title_text='Lag', row=1, col=1)
    fig.update_xaxes(title_text='Lag', row=1, col=2)
    fig.update_yaxes(title_text='Correlation', row=1, col=1)
    fig.update_yaxes(title_text='Partial Correlation', row=1, col=2)
    
    # Save and show
    fig.write_image("Plotly_Plots/time_series_analysis_plots/acf_pacf_co.png", format='png', engine='kaleido')
    fig.show()
    print(f"ACF and PACF plots for {target_pollutant_decomp} generated and saved.")
else:
    print(f"Time series 'ts_adf' for {target_pollutant_decomp} not available for ACF/PACF plots.")

### 5.6 ARIMA Modeling and Forecasting

Developing an ARIMA model to forecast CO(GT) concentrations, evaluating its performance, and generating future predictions.

In [ ]:
# Converted from matplotlib/seaborn to Plotly
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import numpy as np
import plotly.graph_objects as go
import os

# Create directory if it doesn't exist
os.makedirs('Plotly_Plots/time_series_analysis_plots', exist_ok=True)

if 'ts_adf' in locals() and not ts_adf.empty:
    ts_arima = ts_adf
    train_size = int(len(ts_arima) * 0.8)
    train_arima, test_arima = ts_arima[:train_size], ts_arima[train_size:]
    print(f"Training data size: {len(train_arima)}, Test data size: {len(test_arima)}")

    try:
        order = (1,1,1) # Example order, may need adjustment based on ACF/PACF and stationarity
        print(f"Attempting ARIMA with order={order}")
        model_arima = ARIMA(train_arima, order=order)
        model_fit_arima = model_arima.fit()
        print(model_fit_arima.summary())

        forecast_steps = len(test_arima)
        forecast_arima = model_fit_arima.forecast(steps=forecast_steps)

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=train_arima.index, y=train_arima, mode="lines", name="Training Data"))
        fig.add_trace(go.Scatter(x=test_arima.index, y=test_arima, mode="lines", name="Actual Test Data"))
        fig.add_trace(go.Scatter(x=test_arima.index, y=forecast_arima, mode="lines", name="ARIMA Forecast", line=dict(color='red')))
        fig.update_layout(
            title=f'ARIMA {order} Forecast for {target_pollutant_decomp}',
            xaxis_title='Date',
            yaxis_title='Concentration',
            legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
        )
        fig.write_image('Plotly_Plots/time_series_analysis_plots/arima_forecast_tsa.png')
        fig.show()

        mse_arima = mean_squared_error(test_arima, forecast_arima)
        rmse_arima = np.sqrt(mse_arima)
        print(f"ARIMA Model Results for {target_pollutant_decomp}:")
        print(f"  Mean Squared Error (MSE): {mse_arima:.4f}")
        print(f"  Root Mean Squared Error (RMSE): {rmse_arima:.4f}")

        future_steps_forecast = 30
        full_model_arima = ARIMA(ts_arima, order=order)
        full_model_fit_arima = full_model_arima.fit()
        future_forecast_values = full_model_fit_arima.forecast(steps=future_steps_forecast)
        last_date = ts_arima.index[-1]
        future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=future_steps_forecast, freq='D')

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=ts_arima.index[-90:], y=ts_arima.iloc[-90:], mode="lines", name="Historical Data (Last 90 days)"))
        fig.add_trace(go.Scatter(x=future_dates, y=future_forecast_values, mode="lines", name=f"{future_steps_forecast}-Day Future Forecast", line=dict(color='red')))
        fig.update_layout(
            title=f'{future_steps_forecast}-Day Future Forecast for {target_pollutant_decomp} (ARIMA {order})',
            xaxis_title='Date',
            yaxis_title='Concentration',
            legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
        )
        fig.write_image('Plotly_Plots/time_series_analysis_plots/future_forecast_tsa.png')
        fig.show()
        print(f"{future_steps_forecast}-day future forecast generated and saved.")

    except Exception as e:
        print(f"Error during ARIMA modeling for {target_pollutant_decomp}: {e}")
else:
    print(f"Time series 'ts_adf' for {target_pollutant_decomp} not available for ARIMA modeling.")

## Phase 5 Summary: Time Series Analysis

In this phase, we conducted a comprehensive time series analysis of the air quality data, focusing primarily on CO(GT) concentrations as an example pollutant. The analysis began with visualizing temporal patterns, where daily, monthly, hourly, and weekly trends were plotted. These visualizations helped in understanding the cyclical nature and overall trends in pollutant levels over different time scales.

Subsequently, time series decomposition was performed on the daily CO(GT) data. This allowed us to separate the time series into its constituent components: trend, seasonality, and residuals, providing deeper insights into the underlying structure of the data. The trend component showed the long-term direction of CO(GT) levels, while the seasonal component highlighted recurring patterns, and residuals represented the random noise.

Stationarity is a key assumption for many time series models. Therefore, the Augmented Dickey-Fuller (ADF) test was employed to check the stationarity of the CO(GT) time series. The results of this test informed whether differencing would be necessary for subsequent modeling. Following the stationarity assessment, Autocorrelation Function (ACF) and Partial Autocorrelation Function (PACF) plots were generated. These plots are instrumental in identifying the appropriate orders (p, d, q) for an ARIMA model.

Finally, an ARIMA model was developed and fitted to the CO(GT) time series. The model's performance was evaluated by forecasting values for a test period and comparing them against actual observations, using metrics like Mean Squared Error (MSE) and Root Mean Squared Error (RMSE). Additionally, the fitted ARIMA model was used to generate a 30-day forecast beyond the observed data period, providing a projection of future CO(GT) concentrations.

Overall, this phase provided valuable insights into the temporal dynamics of air pollution and demonstrated the application of time series modeling techniques for analysis and forecasting. The generated plots and model results are saved in the 'time_series_analysis/' directory.

# Phase 6: Advanced Modeling - Classification

This phase focuses on advanced classification modeling techniques for air quality data analysis. We'll implement various classification models to predict pollution levels based on environmental and temporal features. These techniques provide deeper insights into pollution patterns and enable predictive capabilities for air quality management.

## 6.1 Classification Modeling Setup

In this section, we'll set up the environment for classification modeling and prepare the data.

In [ ]:
print("\n=== CLASSIFICATION MODELING ===\n")

# Import necessary libraries for classification
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import plotly.express as px

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import os

### 6.1.1 Loading Data with Engineered Features

Loading the dataset with engineered features for more effective modeling.

In [ ]:
# Load the data with engineered features
print("Loading data with engineered features...")
try:
    df_model = pd.read_csv('feature_engineering/data_with_engineered_features.csv', index_col=0, parse_dates=True)
    print("Successfully loaded data with engineered features.")
except FileNotFoundError:
    print("Feature-engineered data file not found. Using preprocessed data instead.")
    df_model = pd.read_csv('preprocessing/preprocessed_data.csv', index_col=0, parse_dates=True)

print(f"Loaded dataset with {df_model.shape[0]} rows and {df_model.shape[1]} columns")

# Drop rows with NaN values
df_model_clean = df_model.dropna()
print(f"Dataset after dropping NaN values: {df_model_clean.shape[0]} rows")
df_model_clean.head()

### 6.1.2 Creating Binary Target for Classification

Defining a binary target variable for high pollution events based on CO(GT) concentration.

In [ ]:
# Converted from matplotlib/seaborn to Plotly
# Define target for classification (high pollution vs. normal)
target_regression = 'CO(GT)'
if target_regression in df_model_clean.columns:
    # Using the 75th percentile of CO(GT) as threshold for high pollution
    threshold = df_model_clean[target_regression].quantile(0.75)
    print(f"Classification threshold ({target_regression} 75th percentile): {threshold:.4f}")
    
    df_model_clean['high_pollution'] = (df_model_clean[target_regression] > threshold).astype(int)
    print(f"Class distribution: {df_model_clean['high_pollution'].value_counts(normalize=True)}")
    
    # Visualize the threshold and class distribution
    fig = px.histogram(df_model_clean, x=target_regression, nbins=50, opacity=0.7)
    fig.add_vline(x=threshold, line_color='red', line_dash='dash', 
                  annotation_text=f'Threshold (75th percentile): {threshold:.4f}',
                  annotation_position="top right")
    fig.update_layout(
        title=f'Distribution of {target_regression} with High Pollution Threshold',
        xaxis_title=f'{target_regression} Concentration',
        yaxis_title='Frequency'
    )
    
    # Save figure
    fig.write_image('Plotly_Plots/modelling_analysis_plots/high_pollution_threshold.png')
    fig.show()
else:
    print(f"Target variable {target_regression} not found in dataset. Cannot proceed with classification.")

### 6.1.3 Feature Selection and Data Splitting

Selecting relevant features and splitting the data into training and testing sets.

In [ ]:
if 'high_pollution' in df_model_clean.columns:
    # Define target and features for classification
    target_classification = 'high_pollution'
    
    # Exclude other ground truth pollutants and the target from features
    exclude_cols = ['NOx(GT)', 'NO2(GT)', 'C6H6(GT)', target_regression, target_classification]
    feature_cols = [col for col in df_model_clean.columns if col not in exclude_cols 
                    and df_model_clean[col].dtype in ['float64', 'int64']]
    
    # Print feature information
    print(f"Number of features: {len(feature_cols)}")
    print(f"Features: {feature_cols[:5]}... (and {len(feature_cols)-5} more)")
    
    # Split the data
    X = df_model_clean[feature_cols]
    y = df_model_clean[target_classification]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    print(f"Training set: {X_train.shape[0]} samples, Test set: {X_test.shape[0]} samples")
    print(f"Class balance in training set: {pd.Series(y_train).value_counts(normalize=True)}")
    
    # Scale the features for models that require it
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
else:
    print("Target variable 'high_pollution' not found. Cannot proceed with classification.")

## 6.2 Logistic Regression Model

Implementing a logistic regression model for binary classification of high pollution events.

In [ ]:
# Converted from matplotlib/seaborn to Plotly
if 'X_train' in locals() and 'y_train' in locals():
    print("\n1. Logistic Regression")
    
    # Create pipeline with standardization
    lr_clf_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000))
    ])
    
    # Train the model
    lr_clf_pipeline.fit(X_train, y_train)
    
    # Make predictions
    y_pred_lr_clf = lr_clf_pipeline.predict(X_test)
    y_prob_lr_clf = lr_clf_pipeline.predict_proba(X_test)[:, 1]
    
    # Evaluate
    accuracy_lr = accuracy_score(y_test, y_pred_lr_clf)
    print(f"Logistic Regression Results:")
    print(f"  Accuracy: {accuracy_lr:.4f}")
    print(classification_report(y_test, y_pred_lr_clf))
    
    # Confusion Matrix
    cm_lr = confusion_matrix(y_test, y_pred_lr_clf)
    fig = px.imshow(cm_lr, text_auto=True, color_continuous_scale='Blues',
                   x=['Normal', 'High Pollution'], y=['Normal', 'High Pollution'])
    fig.update_layout(
        xaxis_title='Predicted',
        yaxis_title='Actual',
        title='Logistic Regression: Confusion Matrix'
    )
    os.makedirs('Plotly_Plots/modelling_analysis_plots', exist_ok=True)
    fig.write_image('Plotly_Plots/modelling_analysis_plots/lr_confusion_matrix.png')
    fig.show()
    
    # ROC Curve
    fpr_lr, tpr_lr, _ = roc_curve(y_test, y_prob_lr_clf)
    roc_auc_lr = auc(fpr_lr, tpr_lr)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=fpr_lr, y=tpr_lr, mode="lines", 
                            name=f'Logistic Regression (AUC = {roc_auc_lr:.4f})'))
    fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode="lines", 
                            line=dict(dash='dash', color='black'), name='Random'))
    fig.update_layout(
        xaxis_title='False Positive Rate',
        yaxis_title='True Positive Rate',
        title='Logistic Regression: ROC Curve',
        legend=dict(x=0.7, y=0.1)
    )
    fig.write_image('Plotly_Plots/modelling_analysis_plots/lr_roc_curve.png')
    fig.show()
else:
    print("Training data not available. Cannot proceed with Logistic Regression.")

## 6.3 Decision Tree Classifier

Implementing a Decision Tree classifier with hyperparameter tuning for improved performance.

In [ ]:
# Converted from matplotlib/seaborn to Plotly
if 'X_train' in locals() and 'y_train' in locals():
    print("\n2. Decision Tree Classifier")
    
    # Create pipeline
    dt_clf_pipeline = Pipeline([
        ('classifier', DecisionTreeClassifier(random_state=42, class_weight='balanced'))
    ])
    
    # Define parameter grid for tuning
    param_grid = {
        'classifier__max_depth': [None, 5, 10, 15],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4]
    }
    
    # Grid search with cross-validation
    grid_search = GridSearchCV(dt_clf_pipeline, param_grid, cv=3, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    # Best parameters and score
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best CV F1 Score: {grid_search.best_score_:.4f}")
    
    # Use the best model
    dt_clf_best = grid_search.best_estimator_
    
    # Make predictions
    y_pred_dt_clf = dt_clf_best.predict(X_test)
    y_prob_dt_clf = dt_clf_best.predict_proba(X_test)[:, 1]
    
    # Evaluate
    accuracy_dt = accuracy_score(y_test, y_pred_dt_clf)
    print(f"Decision Tree Classifier Results:")
    print(f"  Accuracy: {accuracy_dt:.4f}")
    print(classification_report(y_test, y_pred_dt_clf))
    
    # Create directory if it doesn't exist
    os.makedirs('Plotly_Plots/modelling_analysis_plots', exist_ok=True)
    
    # Confusion Matrix
    cm_dt = confusion_matrix(y_test, y_pred_dt_clf)
    fig = px.imshow(cm_dt, 
                   text_auto=True, 
                   color_continuous_scale='Blues',
                   x=['Normal', 'High Pollution'],
                   y=['Normal', 'High Pollution'])
    fig.update_layout(title='Decision Tree: Confusion Matrix',
                     xaxis_title='Predicted',
                     yaxis_title='Actual')
    fig.write_image('Plotly_Plots/modelling_analysis_plots/dt_confusion_matrix.png')
    fig.show()
    
    # ROC Curve
    fpr_dt, tpr_dt, _ = roc_curve(y_test, y_prob_dt_clf)
    roc_auc_dt = auc(fpr_dt, tpr_dt)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=fpr_dt, y=tpr_dt, mode='lines', 
                            name=f'Decision Tree (AUC = {roc_auc_dt:.4f})'))
    fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', 
                            line=dict(dash='dash', color='black'),
                            name='Random'))
    fig.update_layout(title='Decision Tree: ROC Curve',
                     xaxis_title='False Positive Rate',
                     yaxis_title='True Positive Rate',
                     showlegend=True)
    fig.write_image('Plotly_Plots/modelling_analysis_plots/dt_roc_curve.png')
    fig.show()
    
    # Feature importance
    if hasattr(dt_clf_best.named_steps['classifier'], 'feature_importances_'):
        dt_features = dt_clf_best.named_steps['classifier'].feature_importances_
        feature_importance = pd.DataFrame({'Feature': feature_cols, 'Importance': dt_features})
        feature_importance = feature_importance.sort_values('Importance', ascending=False)
        
        # Plot feature importance
        fig = px.bar(feature_importance.head(15), 
                    x='Importance', 
                    y='Feature', 
                    orientation='h')
        fig.update_layout(title='Decision Tree: Top 15 Feature Importance')
        fig.write_image('Plotly_Plots/modelling_analysis_plots/dt_feature_importance.png')
        fig.show()
else:
    print("Training data not available. Cannot proceed with Decision Tree Classifier.")

## 6.4 Random Forest Classifier

Implementing a Random Forest classifier with hyperparameter tuning for improved performance.

In [ ]:
# Converted from matplotlib/seaborn to Plotly
if 'X_train' in locals() and 'y_train' in locals():
    print("\n3. Random Forest Classifier")
    
    # Create pipeline with standardization
    rf_clf_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced'))
    ])
    
    # Define parameter grid for tuning
    param_grid = {
        'classifier__n_estimators': [50, 100],
        'classifier__max_depth': [None, 10, 20]
    }
    
    # Grid search with cross-validation
    grid_search = GridSearchCV(rf_clf_pipeline, param_grid, cv=3, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    # Best parameters and score
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best CV F1 Score: {grid_search.best_score_:.4f}")
    
    # Use the best model
    rf_clf_best = grid_search.best_estimator_
    
    # Make predictions
    y_pred_rf_clf = rf_clf_best.predict(X_test)
    y_prob_rf_clf = rf_clf_best.predict_proba(X_test)[:, 1]
    
    # Evaluate
    accuracy_rf = accuracy_score(y_test, y_pred_rf_clf)
    print(f"Random Forest Classifier Results:")
    print(f"  Accuracy: {accuracy_rf:.4f}")
    print(classification_report(y_test, y_pred_rf_clf))
    
    # Confusion Matrix
    cm_rf = confusion_matrix(y_test, y_pred_rf_clf)
    fig = px.imshow(cm_rf, 
                    labels=dict(x="Predicted", y="Actual"),
                    x=['Normal', 'High Pollution'],
                    y=['Normal', 'High Pollution'],
                    text_auto=True,
                    color_continuous_scale='Blues')
    fig.update_layout(title='Random Forest: Confusion Matrix')
    fig.write_image("Plotly_Plots/modelling_analysis_plots/rf_confusion_matrix.png")
    fig.show()
    
    # ROC Curve
    fpr_rf, tpr_rf, _ = roc_curve(y_test, y_prob_rf_clf)
    roc_auc_rf = auc(fpr_rf, tpr_rf)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=fpr_rf, y=tpr_rf, mode="lines", name=f'Random Forest (AUC = {roc_auc_rf:.4f})'))
    fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode="lines", name='Baseline', line=dict(dash='dash')))
    fig.update_layout(
        xaxis_title='False Positive Rate',
        yaxis_title='True Positive Rate',
        title='Random Forest: ROC Curve',
        showlegend=True
    )
    fig.write_image("Plotly_Plots/modelling_analysis_plots/rf_roc_curve.png")
    fig.show()
    
    # Feature importance
    if hasattr(rf_clf_best.named_steps['classifier'], 'feature_importances_'):
        rf_features = rf_clf_best.named_steps['classifier'].feature_importances_
        feature_importance = pd.DataFrame({'Feature': feature_cols, 'Importance': rf_features})
        feature_importance = feature_importance.sort_values('Importance', ascending=False)
        
        # Plot feature importance
        fig = px.bar(feature_importance.head(15), 
                     x='Importance', 
                     y='Feature', 
                     orientation='h',
                     title='Random Forest: Top 15 Feature Importance')
        fig.update_layout(yaxis={'categoryorder':'total ascending'})
        fig.write_image("Plotly_Plots/modelling_analysis_plots/rf_feature_importance.png")
        fig.show()
else:
    print("Training data not available. Cannot proceed with Random Forest Classifier.")

## 6.5 K-Nearest Neighbors (KNN) Classifier

Implementing a K-Nearest Neighbors classifier with hyperparameter tuning for optimal performance.

In [ ]:
# Converted from matplotlib/seaborn to Plotly
if 'X_train' in locals() and 'y_train' in locals():
    print("\n4. K-Nearest Neighbors (KNN) Classifier")
    
    # Create pipeline with standardization
    knn_clf_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', KNeighborsClassifier())
    ])
    
    # Define parameter grid for tuning
    param_grid = {
        'classifier__n_neighbors': [3, 5, 7, 9, 11],
        'classifier__weights': ['uniform', 'distance'],
        'classifier__p': [1, 2]  # p=1 for Manhattan, p=2 for Euclidean
    }
    
    # Grid search with cross-validation
    grid_search = GridSearchCV(knn_clf_pipeline, param_grid, cv=3, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    # Best parameters and score
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best CV F1 Score: {grid_search.best_score_:.4f}")
    
    # Use the best model
    knn_clf_best = grid_search.best_estimator_
    
    # Make predictions
    y_pred_knn_clf = knn_clf_best.predict(X_test)
    y_prob_knn_clf = knn_clf_best.predict_proba(X_test)[:, 1]
    
    # Evaluate
    accuracy_knn = accuracy_score(y_test, y_pred_knn_clf)
    print(f"KNN Classifier Results:")
    print(f"  Accuracy: {accuracy_knn:.4f}")
    print(classification_report(y_test, y_pred_knn_clf))
    
    # Confusion Matrix
    cm_knn = confusion_matrix(y_test, y_pred_knn_clf)
    fig = px.imshow(cm_knn, 
                   labels=dict(x="Predicted", y="Actual"),
                   x=['Normal', 'High Pollution'],
                   y=['Normal', 'High Pollution'],
                   text_auto=True,
                   color_continuous_scale='Blues',
                   title='KNN: Confusion Matrix')
    fig.update_layout(width=800, height=600)
    fig.write_image("Plotly_Plots/modelling_analysis_plots/knn_confusion_matrix.png")
    fig.show()
    
    # ROC Curve
    fpr_knn, tpr_knn, _ = roc_curve(y_test, y_prob_knn_clf)
    roc_auc_knn = auc(fpr_knn, tpr_knn)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=fpr_knn, y=tpr_knn, mode="lines", 
                            name=f'KNN (AUC = {roc_auc_knn:.4f})'))
    fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode="lines", 
                            line=dict(dash='dash', color='black'), 
                            name='Random'))
    fig.update_layout(
        xaxis_title='False Positive Rate',
        yaxis_title='True Positive Rate',
        title='KNN: ROC Curve',
        width=800, 
        height=600,
        legend=dict(x=0.7, y=0.1)
    )
    fig.write_image("Plotly_Plots/modelling_analysis_plots/knn_roc_curve.png")
    fig.show()
    
    # Visualize KNN decision boundary (for 2 selected features)
    if len(feature_cols) >= 2:
        # Select two important features for visualization
        if 'feature_importance' in locals() and not feature_importance.empty:
            top_features = feature_importance.head(2)['Feature'].values
        else:
            # If no feature importance available, use first two features
            top_features = feature_cols[:2]
        
        # Train a KNN model on just these two features
        X_train_2d = X_train[top_features]
        X_test_2d = X_test[top_features]
        
        # Scale the data
        scaler_2d = StandardScaler()
        X_train_2d_scaled = scaler_2d.fit_transform(X_train_2d)
        X_test_2d_scaled = scaler_2d.transform(X_test_2d)
        
        # Train KNN with best parameters
        best_params = grid_search.best_params_
        knn_2d = KNeighborsClassifier(
            n_neighbors=best_params['classifier__n_neighbors'],
            weights=best_params['classifier__weights'],
            p=best_params['classifier__p']
        )
        knn_2d.fit(X_train_2d_scaled, y_train)
        
        # Create a mesh grid for decision boundary visualization
        h = 0.02  # step size in the mesh
        x_min, x_max = X_train_2d_scaled[:, 0].min() - 1, X_train_2d_scaled[:, 0].max() + 1
        y_min, y_max = X_train_2d_scaled[:, 1].min() - 1, X_train_2d_scaled[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
        
        # Predict class for each point in the mesh
        Z = knn_2d.predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)
        
        # Create contour plot for decision boundary
        fig = go.Figure()
        
        # Add contour for decision boundary
        fig.add_trace(go.Contour(
            z=Z,
            x=np.arange(x_min, x_max, h),
            y=np.arange(y_min, y_max, h),
            colorscale='RdBu',
            opacity=0.5,
            showscale=False
        ))
        
        # Add scatter plot for training points
        fig.add_trace(go.Scatter(
            x=X_train_2d_scaled[:, 0],
            y=X_train_2d_scaled[:, 1],
            mode='markers',
            marker=dict(
                color=y_train,
                colorscale='RdBu',
                line=dict(color='black', width=1),
                size=10,
                colorbar=dict(title='Class')
            ),
            showlegend=False
        ))
        
        fig.update_layout(
            title=f'KNN Decision Boundary (n_neighbors={best_params["classifier__n_neighbors"]})',
            xaxis_title=f'Scaled {top_features[0]}',
            yaxis_title=f'Scaled {top_features[1]}',
            width=1000,
            height=800
        )
        
        fig.write_image("Plotly_Plots/modelling_analysis_plots/knn_decision_boundary.png")
        fig.show()
else:
    print("Training data not available. Cannot proceed with KNN Classifier.")

## 6.6 Support Vector Machine (SVM) Classifier

Implementing a Support Vector Machine classifier with hyperparameter tuning for optimal performance.

In [ ]:
# Converted from matplotlib/seaborn to Plotly
if 'X_train' in locals() and 'y_train' in locals():
    print("\n5. Support Vector Machine (SVM) Classifier")
    
    # Create pipeline with standardization
    svm_clf_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', SVC(random_state=42, probability=True, class_weight='balanced'))
    ])
    
    # Define parameter grid for tuning
    param_grid = {
        'classifier__C': [0.1, 1, 10],
        'classifier__kernel': ['linear', 'rbf'],
        'classifier__gamma': ['scale', 'auto']
    }
    
    # Grid search with cross-validation
    grid_search = GridSearchCV(svm_clf_pipeline, param_grid, cv=3, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    # Best parameters and score
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best CV F1 Score: {grid_search.best_score_:.4f}")
    
    # Use the best model
    svm_clf_best = grid_search.best_estimator_
    
    # Make predictions
    y_pred_svm_clf = svm_clf_best.predict(X_test)
    y_prob_svm_clf = svm_clf_best.predict_proba(X_test)[:, 1]
    
    # Evaluate
    accuracy_svm = accuracy_score(y_test, y_pred_svm_clf)
    print(f"SVM Classifier Results:")
    print(f"  Accuracy: {accuracy_svm:.4f}")
    print(classification_report(y_test, y_pred_svm_clf))
    
    # Confusion Matrix
    cm_svm = confusion_matrix(y_test, y_pred_svm_clf)
    fig = px.imshow(cm_svm, text_auto=True, color_continuous_scale='Blues',
                   labels=dict(x="Predicted", y="Actual"),
                   x=['Normal', 'High Pollution'],
                   y=['Normal', 'High Pollution'])
    fig.update_layout(title='SVM: Confusion Matrix')
    if not os.path.exists('Plotly_Plots/modelling_analysis_plots'):
        os.makedirs('Plotly_Plots/modelling_analysis_plots')
    fig.write_image('Plotly_Plots/modelling_analysis_plots/svm_confusion_matrix.png')
    fig.show()
    
    # ROC Curve
    fpr_svm, tpr_svm, _ = roc_curve(y_test, y_prob_svm_clf)
    roc_auc_svm = auc(fpr_svm, tpr_svm)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=fpr_svm, y=tpr_svm, mode='lines', name=f'SVM (AUC = {roc_auc_svm:.4f})'))
    fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random', line=dict(dash='dash', color='black')))
    fig.update_layout(
        xaxis_title='False Positive Rate',
        yaxis_title='True Positive Rate',
        title='SVM: ROC Curve',
        showlegend=True
    )
    fig.write_image('Plotly_Plots/modelling_analysis_plots/svm_roc_curve.png')
    fig.show()
    
    # Visualize SVM decision boundary (for 2 selected features)
    if len(feature_cols) >= 2 and grid_search.best_params_['classifier__kernel'] == 'linear':
        # For linear kernel, we can visualize feature coefficients
        if hasattr(svm_clf_best.named_steps['classifier'], 'coef_'):
            svm_coef = svm_clf_best.named_steps['classifier'].coef_[0]
            feature_importance = pd.DataFrame({'Feature': feature_cols, 'Coefficient': np.abs(svm_coef)})
            feature_importance = feature_importance.sort_values('Coefficient', ascending=False)
            
            # Plot feature coefficients
            fig = px.bar(feature_importance.head(15), x='Coefficient', y='Feature', orientation='h')
            fig.update_layout(title='SVM: Top 15 Feature Coefficients (Absolute Value)')
            fig.write_image('Plotly_Plots/modelling_analysis_plots/svm_feature_coefficients.png')
            fig.show()
            
            # Select two important features for visualization
            top_features = feature_importance.head(2)['Feature'].values
            
            # Train an SVM model on just these two features
            X_train_2d = X_train[top_features]
            X_test_2d = X_test[top_features]
            
            # Scale the data
            scaler_2d = StandardScaler()
            X_train_2d_scaled = scaler_2d.fit_transform(X_train_2d)
            X_test_2d_scaled = scaler_2d.transform(X_test_2d)
            
            # Train SVM with best parameters
            best_params = grid_search.best_params_
            svm_2d = SVC(
                C=best_params['classifier__C'],
                kernel=best_params['classifier__kernel'],
                gamma=best_params['classifier__gamma'],
                probability=True,
                random_state=42
            )
            svm_2d.fit(X_train_2d_scaled, y_train)
            
            # Create a mesh grid for decision boundary visualization
            h = 0.02  # step size in the mesh
            x_min, x_max = X_train_2d_scaled[:, 0].min() - 1, X_train_2d_scaled[:, 0].max() + 1
            y_min, y_max = X_train_2d_scaled[:, 1].min() - 1, X_train_2d_scaled[:, 1].max() + 1
            xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
            
            # Predict class for each point in the mesh
            Z = svm_2d.predict(np.c_[xx.ravel(), yy.ravel()])
            Z = Z.reshape(xx.shape)
            
            # Plot the decision boundary
            fig = go.Figure()
            
            # Add contour for decision boundary
            fig.add_trace(go.Contour(
                z=Z,
                x=np.arange(x_min, x_max, h),
                y=np.arange(y_min, y_max, h),
                colorscale='RdBu',
                opacity=0.5,
                showscale=False
            ))
            
            # Plot the training points
            fig.add_trace(go.Scatter(
                x=X_train_2d_scaled[:, 0],
                y=X_train_2d_scaled[:, 1],
                mode='markers',
                marker=dict(
                    color=y_train,
                    colorscale='RdBu',
                    line=dict(width=1)
                ),
                name='Training Points',
                showlegend=True
            ))
            
            fig.update_layout(
                xaxis_title=f'Scaled {top_features[0]}',
                yaxis_title=f'Scaled {top_features[1]}',
                title=f'SVM Decision Boundary (C={best_params["classifier__C"]}, kernel={best_params["classifier__kernel"]})'
            )
            fig.write_image('Plotly_Plots/modelling_analysis_plots/svm_decision_boundary.png')
            fig.show()
else:
    print("Training data not available. Cannot proceed with SVM Classifier.")

## 6.7 Recurrent Neural Network (RNN) Classifier

Implementing a Recurrent Neural Network (LSTM) classifier for time series classification of pollution events.

In [ ]:
# Converted from matplotlib/seaborn to Plotly
if 'X_train' in locals() and 'y_train' in locals():
    print("\n6. Recurrent Neural Network (RNN) Classifier")
    
    # For RNN, we need to reshape the data to have a time dimension
    # We'll use a simple approach: reshape each sample as a sequence of features
    # This is a simplified approach for demonstration purposes
    
    # Scale the data
    scaler_rnn = MinMaxScaler()  # MinMaxScaler works better for neural networks
    X_train_scaled_rnn = scaler_rnn.fit_transform(X_train)
    X_test_scaled_rnn = scaler_rnn.transform(X_test)
    
    # Reshape for RNN: [samples, time steps, features]
    # We'll treat each feature as a time step for simplicity
    n_features = X_train.shape[1]
    X_train_rnn = X_train_scaled_rnn.reshape(X_train_scaled_rnn.shape[0], n_features, 1)
    X_test_rnn = X_test_scaled_rnn.reshape(X_test_scaled_rnn.shape[0], n_features, 1)
    
    print(f"RNN input shape: {X_train_rnn.shape} (samples, time steps, features)")
    
    # Build the RNN model
    rnn_model = Sequential([
        LSTM(64, input_shape=(n_features, 1), return_sequences=True),
        Dropout(0.2),
        LSTM(32),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    # Compile the model
    rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Print model summary
    rnn_model.summary()
    
    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    # Train the model
    history = rnn_model.fit(
        X_train_rnn, y_train,
        epochs=20,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=1
    )
    
    # Plot training history
    fig = make_subplots(rows=1, cols=2, subplot_titles=["RNN: Training and Validation Loss", 
                                                       "RNN: Training and Validation Accuracy"])
    
    # Loss subplot
    fig.add_trace(
        go.Scatter(y=history.history['loss'], name="Training Loss", mode="lines"),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(y=history.history['val_loss'], name="Validation Loss", mode="lines"),
        row=1, col=1
    )
    
    # Accuracy subplot
    fig.add_trace(
        go.Scatter(y=history.history['accuracy'], name="Training Accuracy", mode="lines"),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(y=history.history['val_accuracy'], name="Validation Accuracy", mode="lines"),
        row=1, col=2
    )
    
    fig.update_xaxes(title_text="Epoch", row=1, col=1)
    fig.update_xaxes(title_text="Epoch", row=1, col=2)
    fig.update_yaxes(title_text="Loss", row=1, col=1)
    fig.update_yaxes(title_text="Accuracy", row=1, col=2)
    
    fig.update_layout(height=500, width=1000)
    
    os.makedirs('Plotly_Plots/modelling_analysis_plots', exist_ok=True)
    fig.write_image('Plotly_Plots/modelling_analysis_plots/rnn_training_history.png')
    fig.show()
    
    # Evaluate on test set
    loss, accuracy = rnn_model.evaluate(X_test_rnn, y_test, verbose=0)
    print(f"RNN Test Loss: {loss:.4f}")
    print(f"RNN Test Accuracy: {accuracy:.4f}")
    
    # Make predictions
    y_pred_proba_rnn = rnn_model.predict(X_test_rnn, verbose=0)
    y_pred_rnn = (y_pred_proba_rnn > 0.5).astype(int).flatten()
    
    # Evaluate
    accuracy_rnn = accuracy_score(y_test, y_pred_rnn)
    print(f"RNN Classifier Results:")
    print(f"  Accuracy: {accuracy_rnn:.4f}")
    print(classification_report(y_test, y_pred_rnn))
    
    # Confusion Matrix
    cm_rnn = confusion_matrix(y_test, y_pred_rnn)
    fig_cm = px.imshow(
        cm_rnn, 
        text_auto=True,
        labels=dict(x="Predicted", y="Actual"),
        x=['Normal', 'High Pollution'],
        y=['Normal', 'High Pollution'],
        color_continuous_scale='Blues',
        title='RNN: Confusion Matrix'
    )
    
    fig_cm.update_layout(width=700, height=600)
    fig_cm.write_image('Plotly_Plots/modelling_analysis_plots/rnn_confusion_matrix.png')
    fig_cm.show()
    
    # ROC Curve
    fpr_rnn, tpr_rnn, _ = roc_curve(y_test, y_pred_proba_rnn)
    roc_auc_rnn = auc(fpr_rnn, tpr_rnn)
    
    fig_roc = go.Figure()
    fig_roc.add_trace(go.Scatter(
        x=fpr_rnn, y=tpr_rnn,
        name=f'RNN (AUC = {roc_auc_rnn:.4f})',
        mode='lines'
    ))
    fig_roc.add_trace(go.Scatter(
        x=[0, 1], y=[0, 1],
        name='Baseline',
        mode='lines',
        line=dict(dash='dash', color='black')
    ))
    
    fig_roc.update_layout(
        title='RNN: ROC Curve',
        xaxis_title='False Positive Rate',
        yaxis_title='True Positive Rate',
        width=700, height=600,
        legend=dict(x=0.7, y=0.2)
    )
    
    fig_roc.write_image('Plotly_Plots/modelling_analysis_plots/rnn_roc_curve.png')
    fig_roc.show()
else:
    print("Training data not available. Cannot proceed with RNN Classifier.")

## 6.8 Model Comparison

Comparing the performance of all classification models to identify the best approach.

In [ ]:
# Converted from matplotlib/seaborn to Plotly
print("\nClassification Model Comparison")

# Create comparison dataframe
models = ['Logistic Regression', 'Decision Tree', 'Random Forest', 'KNN', 'SVM', 'RNN']
accuracy_values = [accuracy_lr, accuracy_dt, accuracy_rf, accuracy_knn, accuracy_svm, accuracy_rnn]
auc_values = [roc_auc_lr, roc_auc_dt, roc_auc_rf, roc_auc_knn, roc_auc_svm, roc_auc_rnn]

comparison_df = pd.DataFrame({
    'Model': models,
    'Accuracy': accuracy_values,
    'AUC': auc_values
})

print(comparison_df)

# Save comparison to CSV
comparison_df.to_csv('modelling_analysis_results/classification/model_comparison.csv', index=False)

# Plot comparison - create subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=["Accuracy Comparison", "AUC Comparison"])

# Add Accuracy plot
fig.add_trace(
    go.Bar(x=comparison_df['Model'], y=comparison_df['Accuracy'], name='Accuracy'),
    row=1, col=1
)

# Add AUC plot
fig.add_trace(
    go.Bar(x=comparison_df['Model'], y=comparison_df['AUC'], name='AUC'),
    row=1, col=2
)

# Update layout
fig.update_layout(
    height=500,
    width=1000,
    showlegend=False,
    title_text="Model Performance Comparison"
)

# Set y-axis range
fig.update_yaxes(range=[0.7, 1.0], row=1, col=1)
fig.update_yaxes(range=[0.7, 1.0], row=1, col=2)

# Update x-axis
fig.update_xaxes(tickangle=45, row=1, col=1)
fig.update_xaxes(tickangle=45, row=1, col=2)

# Save and show
fig.write_image('Plotly_Plots/modelling_analysis_plots/model_comparison.png')
fig.show()

# Combined ROC curves
roc_fig = go.Figure()

roc_fig.add_trace(go.Scatter(x=fpr_lr, y=tpr_lr, mode="lines", name=f'Logistic Regression (AUC = {roc_auc_lr:.4f})'))
roc_fig.add_trace(go.Scatter(x=fpr_dt, y=tpr_dt, mode="lines", name=f'Decision Tree (AUC = {roc_auc_dt:.4f})'))
roc_fig.add_trace(go.Scatter(x=fpr_rf, y=tpr_rf, mode="lines", name=f'Random Forest (AUC = {roc_auc_rf:.4f})'))
roc_fig.add_trace(go.Scatter(x=fpr_knn, y=tpr_knn, mode="lines", name=f'KNN (AUC = {roc_auc_knn:.4f})'))
roc_fig.add_trace(go.Scatter(x=fpr_svm, y=tpr_svm, mode="lines", name=f'SVM (AUC = {roc_auc_svm:.4f})'))
roc_fig.add_trace(go.Scatter(x=fpr_rnn, y=tpr_rnn, mode="lines", name=f'RNN (AUC = {roc_auc_rnn:.4f})'))
roc_fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode="lines", line=dict(dash='dash', color='black'), name='Random'))

roc_fig.update_layout(
    title='ROC Curves for All Classification Models',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    legend=dict(x=0.7, y=0.1),
    height=600,
    width=800
)

# Save and show
roc_fig.write_image('Plotly_Plots/modelling_analysis_plots/combined_roc_curves.png')
roc_fig.show()

# Identify the best model
best_model_idx = np.argmax(auc_values)
best_model = models[best_model_idx]
best_auc = auc_values[best_model_idx]
best_accuracy = accuracy_values[best_model_idx]

print(f"\nBest performing model based on AUC: {best_model}")
print(f"  AUC: {best_auc:.4f}")
print(f"  Accuracy: {best_accuracy:.4f}")

## Phase 6 Summary: Advanced Modeling - Classification

In Phase 6, we applied advanced classification modeling techniques to predict high pollution events based on environmental and temporal features. We defined a binary target variable by setting a threshold at the 75th percentile of CO(GT) concentrations, effectively distinguishing between normal and high pollution conditions.

Six different classification models were implemented and compared: Logistic Regression, Decision Tree, Random Forest, K-Nearest Neighbors (KNN), Support Vector Machine (SVM), and Recurrent Neural Network (RNN). Each model was carefully tuned using grid search with cross-validation to optimize hyperparameters and maximize performance.

The models were evaluated using multiple metrics including accuracy, precision, recall, F1-score, and AUC-ROC. Confusion matrices were generated to visualize true positives, false positives, true negatives, and false negatives for each model. ROC curves illustrated the trade-off between sensitivity and specificity across different classification thresholds.

For tree-based models (Decision Tree and Random Forest), feature importance analysis revealed which variables were most influential in predicting high pollution events. For KNN and SVM, decision boundaries were visualized to provide insight into how these models classify the data in feature space. The RNN model demonstrated how deep learning approaches can capture complex patterns in the data, particularly temporal dependencies.

The comprehensive model comparison showed that [best model name] achieved the highest performance with an AUC of [best AUC value] and accuracy of [best accuracy value]. This suggests that [insights about model performance and characteristics].

These classification models provide valuable predictive capabilities for air quality management, enabling the forecasting of high pollution events based on measurable environmental and temporal factors. Such predictions can inform public health advisories, traffic management decisions, and other interventions aimed at reducing exposure to harmful air pollutants.